In [1]:
%matplotlib inline
import random
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import date

In [2]:
df = pd.read_csv('./L.POINT_train.csv')
df_test = pd.read_csv('./L.POINT_test.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
y = pd.read_csv('./y_train.csv')

In [4]:
df.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,KWD_NM,SEARCH_CNT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,0,6771240,63,578845,1개,필립스(PHILIPS),"81,000",1,17,20180609,...,922,mobile,Gyeonggi-do,Bucheon-si,에어컨 커버,1,아방세 프로믹스 핸드블렌더 HR1672/90,생활/주방가전,주방가전,블랜더
1,0,6771240,63,788068,선택:버닝 [베이지] / 1개,쁘리엘르,"5,500",1,17,20180609,...,922,mobile,Gyeonggi-do,Bucheon-si,에어컨 커버,1,스판 벽걸이 에어컨커버(트라이앵글_82x27x26) 모음 - 블루가든 [블루],침구/수예,수예소품,거실수예소품
2,1,5762174,109,180447,사이즈:L(105) / 1개,퀵실버,"59,000",1,12,20180626,...,"1,661",mobile,Seoul,Seoul,바비브라운,1,퀵실버 남성 루즈핏 래쉬가드 QS579KMT - M(100),시즌스포츠,수영/물놀이,남성수영복
3,1,5753875,94,731145,1개,키엘,"39,000",1,13,20180626,...,620,mobile,Seoul,Seoul,키엘,2,칼렌듈라 딥 클렌징 포밍 페이스 워시 230ml,화장품/뷰티케어,스킨케어,페이셜클렌저
4,1,7417570,114,216947,1개,키엘,"49,000",1,2,20180529,...,860,mobile,Seoul,Seoul,키엘비타민,2,키엘 자외선 차단제 점보 세트,화장품/뷰티케어,선케어,선크림류


In [5]:
len(df['KWD_NM'].unique())

72089

In [6]:
len(df['KWD_NM'].unique())

72089

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948686 entries, 0 to 1948685
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   CLNT_ID          int64  
 1   SESS_ID          int64  
 2   HITS_SEQ         int64  
 3   PD_C             int64  
 4   PD_ADD_NM        object 
 5   PD_BRA_NM        object 
 6   PD_BUY_AM        object 
 7   PD_BUY_CT        object 
 8   SESS_SEQ         int64  
 9   SESS_DT          int64  
 10  TOT_PAG_VIEW_CT  float64
 11  TOT_SESS_HR_V    object 
 12  DVC_CTG_NM       object 
 13  ZON_NM           object 
 14  CITY_NM          object 
 15  KWD_NM           object 
 16  SEARCH_CNT       int64  
 17  PD_NM            object 
 18  CLAC1_NM         object 
 19  CLAC2_NM         object 
 20  CLAC3_NM         object 
dtypes: float64(1), int64(7), object(13)
memory usage: 312.2+ MB


In [8]:
y

,CLNT_ID,LABEL
0,0,F20
1,1,F30
2,6,F20
3,9,F30
4,12,F30
...,...,...
149995,263094,F30
149996,263095,F30
149997,263096,F30
149998,263102,F30


In [9]:
y['LABEL'].unique()

array(['F20', 'F30', 'F40', 'M30', 'M40', 'M20'], dtype=object)

In [10]:
y['LABEL'].value_counts()

F30    59892
F40    51936
F20    17727
M40     9904
M30     7953
M20     2588
Name: LABEL, dtype: int64

In [11]:
weight = np.array([59892/17727, 59892/59892, 59892/51936, 59892/2588, 59892/7953, 59892/9904])
weight

array([ 3.37857506,  1.        ,  1.15318854, 23.14219474,  7.53074312,
        6.04725363])

In [12]:
df = pd.merge(df, y, how='left', on='CLNT_ID')

In [13]:
len(df['CLNT_ID'].unique())

150000

1948686행, 150000 유저 -> 유저당 평균 13회 로그 정보

In [14]:
len(df['PD_C'].unique())

280867

280867개 고유 상품

In [15]:
df.groupby(by='LABEL')['DVC_CTG_NM'].value_counts()

LABEL  DVC_CTG_NM
F20    mobile        150178
       tablet           519
       desktop           18
F30    mobile        794940
       tablet          2417
       desktop           13
F40    mobile        772145
       tablet          5190
       desktop           29
M20    mobile         19259
       tablet           150
       desktop            9
M30    mobile         78795
       tablet           109
       desktop           10
M40    mobile        123945
       tablet           896
       desktop           64
Name: DVC_CTG_NM, dtype: int64

In [16]:
df['ZON_NM'].unique()

array(['Gyeonggi-do', 'Seoul', 'Jeollabuk-do', 'Gwangju', 'Busan',
       'Incheon', 'Gyeongsangnam-do', 'Gyeongsangbuk-do',
       'Chungcheongnam-do', 'Jeollanam-do', 'Daegu', 'Gangwon-do',
       'Ulsan', 'Daejeon', 'Chungcheongbuk-do', 'Jeju-do'], dtype=object)

In [17]:
df['CITY_NM'].unique()

array(['Bucheon-si', 'Seoul', 'Wanju-gun', 'Gwangju', 'Namyangju-si',
       'Guri-si', 'Busan', 'Imsil-gun', 'Gimpo-si', 'Incheon',
       'Gimhae-si', 'Paju-si', 'Sangju-si', 'Yeongi-gun', 'Hwaseong-si',
       'Gongju-si', 'Seongnam-si', 'Andong', 'Jeongeup-si', 'Geoje-si',
       'Gangjin-gun', 'Gumi-si', 'Yeoju-gun', 'Pyeongtaek-si', 'Daegu',
       'Yongin-si', 'Yangsan-si', 'Chuncheon-si', 'Suwon-si', 'Ulsan',
       'Tongyeong-si', 'Haman-gun', 'Goyang-si', 'Daejeon', 'Cheonan-si',
       'Yeosu-si', 'Jeonju-si', 'Taean-gun', 'Chungju-si', 'Uijeongbu-si',
       'Gangneung-si', 'Gyeongju-si', 'Hanam-si', 'Sunchang-gun',
       'Mungyeong-si', 'Gunsan-si', 'Wonju-si', 'Uiwang-si', 'Anseong',
       'Anyang', 'Ansan-si', 'Gunpo-si', 'Sacheon-si', 'Cheongju-si',
       'Miryang-si', 'Gimcheon-si', 'Sokcho-si', 'Yeongju-si', 'Jeju-si',
       'Siheung-si', 'Pohang-si', 'Seosan-si', 'Hapcheon-gun',
       'Yangju-si', 'Taebaek-si', 'Muan-gun', 'Goesan-gun', 'Jindo-gun',
       'Gwan

In [18]:
df['PD_NM']

0                             아방세 프로믹스 핸드블렌더 HR1672/90
1          스판 벽걸이 에어컨커버(트라이앵글_82x27x26) 모음 - 블루가든 [블루]
2                    퀵실버 남성 루즈핏 래쉬가드 QS579KMT - M(100)
3                           칼렌듈라 딥 클렌징 포밍 페이스 워시 230ml
4                                     키엘 자외선 차단제 점보 세트
                              ...                     
1948681         돌핀 배색 올인원 수영복[71SW77831] - 핑크(P) / 110
1948682         돌핀 배색 올인원 수영복[71SW77831] - 핑크(P) / 110
1948683         돌핀 배색 올인원 수영복[71SW77831] - 핑크(P) / 110
1948684         돌핀 배색 올인원 수영복[71SW77831] - 핑크(P) / 110
1948685        여아 돌고래 배색 래쉬가드[71SW50831] - 핑크(P) / 105
Name: PD_NM, Length: 1948686, dtype: object

In [19]:
len(df['PD_NM'].unique())

276002

num(PD_C) != num(PD_NM)

In [20]:
len(df['CLAC1_NM'].unique()), len(df['CLAC2_NM'].unique()), len(df['CLAC3_NM'].unique())

(37, 128, 883)

대분류 37개, 중분류 128개, 소분류 883개

In [21]:
df['KWD_NM']

0           에어컨 커버
1           에어컨 커버
2            바비브라운
3               키엘
4            키엘비타민
            ...   
1948681       키즈샌들
1948682        갭키즈
1948683      팁토이조이
1948684    갭키즈 수영복
1948685     갭키즈 여아
Name: KWD_NM, Length: 1948686, dtype: object

In [22]:
df['KWD_NM'].isna().sum()

0

In [23]:
df['SEARCH_CNT']

0          1
1          1
2          1
3          2
4          2
          ..
1948681    1
1948682    1
1948683    1
1948684    1
1948685    1
Name: SEARCH_CNT, Length: 1948686, dtype: int64

In [24]:
clnt_id = y['CLNT_ID'].values

In [25]:
clnt_id

array([     0,      1,      6, ..., 263096, 263102, 263103], dtype=int64)

In [26]:
df['PD_BUY_CT'] = df['PD_BUY_CT'].astype('string')
df['TOT_SESS_HR_V'] = df['TOT_SESS_HR_V'].astype('string')

df['PD_BUY_AM'] = df['PD_BUY_AM'].map(lambda x: x.replace(',', ''))
df['PD_BUY_CT'] = df['PD_BUY_CT'].map(lambda x: x.replace(',', ''))
df['TOT_SESS_HR_V'] = df['TOT_SESS_HR_V'].map(lambda x: x.replace(',', ''))

In [27]:
df['PD_BUY_AM'] = df['PD_BUY_AM'].astype('int')
df['PD_BUY_CT'] = df['PD_BUY_CT'].astype('int')
df['TOT_SESS_HR_V'] = df['TOT_SESS_HR_V'].astype('int')

In [28]:
df['SESS_DT'] = df['SESS_DT'].map(lambda x: date.fromisoformat(str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:]))

In [29]:
temp_df = df[df['CLNT_ID'] == clnt_id[4]]
temp_df = temp_df.sort_values(by=['SESS_DT', 'HITS_SEQ'])
temp_df = temp_df[~temp_df.duplicated(subset=['SESS_ID', 'HITS_SEQ'], keep='last')]
temp_df

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,DVC_CTG_NM,ZON_NM,CITY_NM,KWD_NM,SEARCH_CNT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,LABEL
48,12,4738380,47,178471,사이즈:18M / 1개,갭 키즈,10000,1,175,2018-07-12,...,mobile,Seoul,Seoul,베베드피노 오버롤,1,베이비 여아 화이트 블루머 5238234509001 - 24M,속옷/양말/홈웨어,유아동양말류,유아동타이즈,F30
44,12,4551726,32,233182,색상:BLUE|사이즈:85 / 1개,베베드피노,39000,1,189,2018-07-16,...,mobile,Gyeonggi-do,Namyangju-si,뚜아후아 귀걸이,4,베이비 스트라이프 러플 바디수트BP8216222 - BLUE / 85,유아동의류,유아의류전신,영유아점프수트/오버롤,F30
57,12,4130706,104,141349,모델명:01>MS4388아이보리|사이즈:39 / 1개,클립,79000,1,219,2018-07-22,...,mobile,Gyeonggi-do,Guri-si,코치,1,베스트샌들 MS4388 Malou_Color line Webbing 아이보리 외 1...,패션잡화,여성화,여성샌들,F30
55,12,1776250,36,72185,색상:코랄|사이즈:S(6~12m) / 1개,해피프린스,4560,1,313,2018-09-01,...,mobile,Gyeonggi-do,Guri-si,해피프린스,4,나리앙 니삭스 - 코랄 / M(12~24m),속옷/양말/홈웨어,유아동양말류,유아동일반양말,F30
60,12,1011134,50,536257,색상:베이지|사이즈:S / 1개,해피프린스,990,1,345,2018-09-13,...,mobile,Gyeonggi-do,Namyangju-si,해피프린스 양말,1,폴리지 삭스 - 블루 / S,속옷/양말/홈웨어,유아동양말류,유아동일반양말,F30
59,12,1011134,58,102618,1개,테팔,15900,1,345,2018-09-13,...,mobile,Gyeonggi-do,Namyangju-si,해피프린스 양말,1,그래픽 패스포트 아이 러브 뉴욕 프라이팬 26cm,식기/조리기구,조리기구,프라이팬,F30


In [30]:
def calc_avg_shopping_interval(df):
    mean_shopping_interval = 0
    for i in range(len(df)-1):
        mean_shopping_interval += (df['SESS_DT'].iloc[i+1] - df['SESS_DT'].iloc[i]).days
    if len(df)-1 == 0:
        mean_shopping_interval = 183
    else:
        mean_shopping_interval = mean_shopping_interval / (len(df)-1)
    return mean_shopping_interval

In [31]:
def calc_avg_total_price_ct(df):
    price = 0
    cnt = 0
    for i in range(len(df)):
        price += df['PD_BUY_AM'].iloc[i] * df['PD_BUY_CT'].iloc[i]
        cnt += df['PD_BUY_CT'].iloc[i]
    return price/cnt, price, cnt/len(df), cnt

#### 피처 생성 루프 코드

In [32]:
num_shoppings = []
avg_prices = []
total_prices = []
avg_cts = []
total_cts = []
avg_sess_views = []
total_sess_views = []
avg_sess_hrs = []
total_sess_hrs = []
avg_shopping_intervals = []
main_devices = []
pd_cs = []
clac1_nms = []
clac2_nms = []
clac3_nms = []

for i in tqdm(range(len(clnt_id))):
    temp_df = df[df['CLNT_ID'] == clnt_id[i]]
    temp_df = temp_df.sort_values(by=['SESS_DT', 'HITS_SEQ', 'PD_C'])
    temp_df = temp_df[~temp_df.duplicated(subset=['SESS_ID', 'HITS_SEQ', 'PD_C'], keep='last')]
    
    num_shopping = len(temp_df)
    avg_price, total_price, avg_ct, total_ct = calc_avg_total_price_ct(temp_df)
    avg_sess_view = temp_df['TOT_PAG_VIEW_CT'].values.mean()
    total_sess_view = temp_df['TOT_PAG_VIEW_CT'].values.sum()
    avg_sess_hr = temp_df['TOT_SESS_HR_V'].values.mean()
    total_sess_hr = temp_df['TOT_SESS_HR_V'].values.sum()
    avg_shopping_interval = calc_avg_shopping_interval(temp_df)
    main_device = scipy.stats.mode(temp_df['DVC_CTG_NM'].values).mode[0]
    pd_c = temp_df['PD_C'].values
    clac1_nm = temp_df['CLAC1_NM'].values
    clac2_nm = temp_df['CLAC2_NM'].values
    clac3_nm = temp_df['CLAC3_NM'].values
    
    num_shoppings.append(num_shopping)
    avg_prices.append(avg_price)
    total_prices.append(total_price)
    avg_cts.append(avg_ct)
    total_cts.append(total_ct)
    avg_sess_views.append(avg_sess_view)
    total_sess_views.append(total_sess_view)
    avg_sess_hrs.append(avg_sess_hr)
    total_sess_hrs.append(total_sess_hr)
    avg_shopping_intervals.append(avg_shopping_interval)
    main_devices.append(main_device)
    pd_cs.append(pd_c)
    clac1_nms.append(clac1_nm)
    clac2_nms.append(clac2_nm)
    clac3_nms.append(clac3_nm)

100%|█████████████████████████████████████████████████████████████████████████| 150000/150000 [15:52<00:00, 157.46it/s]


In [33]:
data = pd.DataFrame([clnt_id, num_shoppings, avg_prices, total_prices, avg_cts, total_cts, avg_sess_views, 
                     total_sess_views, avg_sess_hrs, total_sess_hrs, avg_shopping_intervals, main_devices, 
                     pd_cs, clac1_nms, clac2_nms, clac3_nms, y['LABEL']]).T

In [34]:
data.columns = ['clnt_id', 'num_shopping', 'avg_price', 'total_price', 'avg_ct', 'total_ct', 'avg_sess_view', 
                'total_sess_view', 'avg_sess_hr', 'total_sess_hr', 'avg_shopping_interval', 'main_device', 
                'pd_c', 'clac1_nm', 'clac2_nm', 'clac3_nm', 'label']

In [35]:
data

,clnt_id,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,avg_shopping_interval,main_device,pd_c,clac1_nm,clac2_nm,clac3_nm,label
0,0,2,43250,86500,1,2,59,118,922,1844,0,mobile,"[578845, 788068]","[생활/주방가전, 침구/수예]","[주방가전, 수예소품]","[블랜더, 거실수예소품]",F20
1,1,9,77777.8,700000,1,9,132.333,1191,1311.11,11800,3.625,mobile,"[216947, 236174, 1965, 190233, 731145, 180447,...","[화장품/뷰티케어, 화장품/뷰티케어, 건강식품, 화장품/뷰티케어, 화장품/뷰티케어,...","[선케어, 스킨케어, 홍삼/인삼가공식품, 스킨케어, 스킨케어, 수영/물놀이, 메이크...","[선크림류, 에센스/세럼, 홍삼액, 에센스/세럼, 페이셜클렌저, 남성수영복, BB/...",F30
2,6,4,24225,96900,1,4,21.75,87,297.25,1189,5.66667,mobile,"[554217, 248358, 506337, 506359]","[스포츠패션, 속옷/양말/홈웨어, 속옷/양말/홈웨어, 속옷/양말/홈웨어]","[여성스포츠화, 여성속옷, 여성속옷, 여성속옷]","[여성스포츠샌들/슬리퍼, 여성속옷세트, 여성팬티, 브래지어]",F20
3,9,2,10550,21100,1,2,249,498,5049,10098,0,mobile,"[436275, 578537]","[속옷/양말/홈웨어, 속옷/양말/홈웨어]","[유아동속옷, 유아동속옷]","[유아동팬티, 유아동팬티]",F30
4,12,16,14325.6,229210,1,16,144.938,2319,4187.25,66996,4.2,mobile,"[178471, 233182, 141349, 64916, 72185, 489942,...","[속옷/양말/홈웨어, 유아동의류, 패션잡화, 패션잡화, 속옷/양말/홈웨어, 속옷/양...","[유아동양말류, 유아의류전신, 여성화, 모자, 유아동양말류, 유아동양말류, 모자, ...","[유아동타이즈, 영유아점프수트/오버롤, 여성샌들, 아동모, 유아동일반양말, 유아동일...",F30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,263094,1,10000,10000,1,1,66,66,513,513,183,mobile,[536890],[패션잡화],[여성화],[여성플랫],F30
149996,263095,2,122000,244000,1,2,220.5,441,1828,3656,83,mobile,"[411059, 741695]","[유아동의류, 화장품/뷰티케어]","[여아의류아우터, 스킨케어]","[여아점퍼, 스킨케어세트]",F30
149997,263096,3,28500,85500,1,3,256,768,4237,12711,0,mobile,"[406269, 592279, 592285]","[스포츠패션, 스포츠패션, 스포츠패션]","[여성스포츠화, 남성일반스포츠의류, 남성일반스포츠의류]","[여성런닝/트레이닝화, 남성일반스포츠바지, 남성일반스포츠바지]",F30
149998,263102,1,1080,1080,1,1,188,188,1812,1812,183,mobile,[785281],[문구/사무용품],[필기도구],[볼펜],F30


In [36]:
pd_c_dic = {}
for i, nm in enumerate(pd.concat([df, df_test], axis=0)['PD_C'].unique()):
    pd_c_dic[nm] = i

In [37]:
pd_c_dic

{578845: 0,
 788068: 1,
 180447: 2,
 731145: 3,
 216947: 4,
 491876: 5,
 1965: 6,
 457181: 7,
 190233: 8,
 236174: 9,
 248358: 10,
 506359: 11,
 506337: 12,
 554217: 13,
 436275: 14,
 578537: 15,
 536257: 16,
 141349: 17,
 535684: 18,
 75955: 19,
 29069: 20,
 41283: 21,
 489942: 22,
 760527: 23,
 233182: 24,
 178471: 25,
 64916: 26,
 535442: 27,
 535601: 28,
 64933: 29,
 72185: 30,
 102618: 31,
 809170: 32,
 837392: 33,
 765673: 34,
 660999: 35,
 804: 36,
 375713: 37,
 133407: 38,
 742158: 39,
 742089: 40,
 484590: 41,
 641183: 42,
 68947: 43,
 842488: 44,
 225207: 45,
 682061: 46,
 569802: 47,
 832202: 48,
 221420: 49,
 826313: 50,
 276275: 51,
 140325: 52,
 694453: 53,
 811116: 54,
 585440: 55,
 700995: 56,
 7893: 57,
 733585: 58,
 185099: 59,
 722909: 60,
 611777: 61,
 425921: 62,
 499940: 63,
 549195: 64,
 220714: 65,
 218301: 66,
 626454: 67,
 473112: 68,
 739681: 69,
 746888: 70,
 657620: 71,
 478990: 72,
 676622: 73,
 685372: 74,
 186166: 75,
 455026: 76,
 454153: 77,
 839324: 7

In [38]:
data['pd_c'] = data['pd_c'].map(lambda x: [pd_c_dic[nm] for nm in x])

- 클라이언트 별 각 대 중 소 분류 검색 누적 숫자 => matrix 변환

In [39]:
clac1_nm_dic = {}
for i, nm in enumerate(df['CLAC1_NM'].unique()):
    clac1_nm_dic[nm] = i

In [40]:
clac1_nm_dic

{'생활/주방가전': 0,
 '침구/수예': 1,
 '시즌스포츠': 2,
 '화장품/뷰티케어': 3,
 '건강식품': 4,
 '속옷/양말/홈웨어': 5,
 '스포츠패션': 6,
 '패션잡화': 7,
 '주방잡화': 8,
 '유아동의류': 9,
 '식기/조리기구': 10,
 '세제/위생': 11,
 '청소/세탁/욕실용품': 12,
 '출산/육아용품': 13,
 '냉장/세탁가전': 14,
 '퍼스널케어': 15,
 '구기/필드스포츠': 16,
 '원예/애완': 17,
 '음료': 18,
 '여성의류': 19,
 '남성의류': 20,
 '계절가전': 21,
 '냉장식품': 22,
 '아웃도어/레저': 23,
 '냉동식품': 24,
 '가구': 25,
 '완구': 26,
 '헬스/피트니스': 27,
 '축산물': 28,
 '컴퓨터': 29,
 '모바일': 30,
 '문구/사무용품': 31,
 '인테리어/조명': 32,
 '상품권': 33,
 '과일': 34,
 '자동차용품': 35,
 '영상/음향가전': 36}

In [41]:
data['clac1_nm'] = data['clac1_nm'].map(lambda x: [clac1_nm_dic[nm] for nm in x])

In [42]:
clac1_matrix = np.zeros((len(data), len(clac1_nm_dic)))
for i in range(len(data)):
    for j in data['clac1_nm'][i]:
        clac1_matrix[i, j] += 1

In [43]:
clac1_matrix

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.]])

In [44]:
cols = ['clac1_nm_' + str(i) for i in range(len(clac1_nm_dic))]

In [45]:
clac1_df = pd.DataFrame(clac1_matrix)
clac1_df.columns = cols

In [46]:
clac1_df

,clac1_nm_0,clac1_nm_1,clac1_nm_2,clac1_nm_3,clac1_nm_4,clac1_nm_5,clac1_nm_6,clac1_nm_7,clac1_nm_8,clac1_nm_9,...,clac1_nm_27,clac1_nm_28,clac1_nm_29,clac1_nm_30,clac1_nm_31,clac1_nm_32,clac1_nm_33,clac1_nm_34,clac1_nm_35,clac1_nm_36
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,10.0,0.0,3.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149996,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149997,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [47]:
clac2_nm_dic = {}
for i, nm in enumerate(df['CLAC2_NM'].unique()):
    clac2_nm_dic[nm] = i

In [48]:
clac2_nm_dic

{'주방가전': 0,
 '수예소품': 1,
 '수영/물놀이': 2,
 '스킨케어': 3,
 '선케어': 4,
 '홍삼/인삼가공식품': 5,
 '메이크업': 6,
 '여성속옷': 7,
 '여성스포츠화': 8,
 '유아동속옷': 9,
 '유아동양말류': 10,
 '여성화': 11,
 '조리도구': 12,
 '유아의류전신': 13,
 '모자': 14,
 '조리기구': 15,
 '화장지/티슈': 16,
 '정리용품': 17,
 '유아스킨/바디케어': 18,
 '냉장/냉동고': 19,
 '핸드/풋케어': 20,
 '남성일반스포츠의류': 21,
 '골프': 22,
 '남성지갑': 23,
 '남성스포츠화': 24,
 '애견용품': 25,
 '생수': 26,
 '여성일반스포츠의류': 27,
 '건강진액': 28,
 '남성케어': 29,
 '우산/양산류': 30,
 '스포츠잡화': 31,
 '남성속옷': 32,
 '여성의류상의': 33,
 '여성의류전신': 34,
 '유아동침구': 35,
 '남성의류상의': 36,
 '남성의류하의': 37,
 '여성의류아우터': 38,
 '냉방가전': 39,
 '유아의류상의': 40,
 '여아의류상의': 41,
 '여성위생용품': 42,
 '유아위생용품': 43,
 '영양제': 44,
 '포장반찬': 45,
 '남성등산/아웃도어의류': 46,
 '유아동스포츠화': 47,
 '헤어케어': 48,
 '캠핑': 49,
 '등산': 50,
 '여성가방': 51,
 '유아의류하의': 52,
 '고양이용품': 53,
 '구강케어': 54,
 '남성의류아우터': 55,
 '냉동간편식': 56,
 '수납가구': 57,
 '사무용/학생용가구': 58,
 '수유/이유용품': 59,
 '유아발육용품': 60,
 '유아동화': 61,
 '여성의류하의': 62,
 '교육완구': 63,
 '피트니스': 64,
 '그릇/식기': 65,
 '남성골프의류': 66,
 '여성골프의류': 67,
 '거실가구': 68,
 '닭고기류': 69,
 '남아의류상의': 70,
 '홈웨

In [49]:
data['clac2_nm'] = data['clac2_nm'].map(lambda x: [clac2_nm_dic[nm] for nm in x])

In [50]:
clac2_matrix = np.zeros((len(data), len(clac2_nm_dic)))
for i in range(len(data)):
    for j in data['clac2_nm'][i]:
        clac2_matrix[i, j] += 1

In [51]:
clac2_matrix

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.]])

In [52]:
cols = ['clac2_nm_' + str(i) for i in range(len(clac2_nm_dic))]

In [53]:
clac2_df = pd.DataFrame(clac2_matrix)
clac2_df.columns = cols

In [54]:
clac2_df

,clac2_nm_0,clac2_nm_1,clac2_nm_2,clac2_nm_3,clac2_nm_4,clac2_nm_5,clac2_nm_6,clac2_nm_7,clac2_nm_8,clac2_nm_9,...,clac2_nm_118,clac2_nm_119,clac2_nm_120,clac2_nm_121,clac2_nm_122,clac2_nm_123,clac2_nm_124,clac2_nm_125,clac2_nm_126,clac2_nm_127
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,4.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149996,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
clac3_nm_dic = {}
for i, nm in enumerate(df['CLAC3_NM'].unique()):
    clac3_nm_dic[nm] = i

In [56]:
clac3_nm_dic

{'블랜더': 0,
 '거실수예소품': 1,
 '남성수영복': 2,
 '페이셜클렌저': 3,
 '선크림류': 4,
 '여성비치웨어': 5,
 '홍삼액': 6,
 'BB/파운데이션/컴팩트류': 7,
 '에센스/세럼': 8,
 '여성속옷세트': 9,
 '브래지어': 10,
 '여성팬티': 11,
 '여성스포츠샌들/슬리퍼': 12,
 '유아동팬티': 13,
 '유아동일반양말': 14,
 '여성샌들': 15,
 '유아동타이즈': 16,
 '주방칼/가위': 17,
 '영유아점프수트/오버롤': 18,
 '아동모': 19,
 '프라이팬': 20,
 '롤티슈': 21,
 '플라스틱서랍장': 22,
 '유아용화장품': 23,
 '일반형냉장고': 24,
 '핸드로션/크림': 25,
 '남성스포츠티셔츠': 26,
 '크림/밤/오일': 27,
 '골프공': 28,
 '골프연습장비': 29,
 '립글로즈/틴트': 30,
 '남성일반지갑': 31,
 '남성런닝/트레이닝화': 32,
 '애견주거/실내용품': 33,
 '생수': 34,
 '여성트레이닝복': 35,
 '여성런닝셔츠/캐미솔': 36,
 '채소즙': 37,
 '남성용스킨케어류': 38,
 '3단우산': 39,
 '스포츠가방': 40,
 '남성팬티': 41,
 '여성남방셔츠': 42,
 '여성원피스': 43,
 '유아동이불/이불커버': 44,
 '남성티셔츠': 45,
 '남성캐주얼바지': 46,
 '여성코트': 47,
 '남성청바지': 48,
 '기타여성속옷': 49,
 '기타냉방가전': 50,
 '영유아티셔츠/탑': 51,
 '여아티셔츠/탑': 52,
 '생리대': 53,
 '유아용기저귀': 54,
 '유산균/프로바이오틱스': 55,
 '김치류': 56,
 '남성등산바지': 57,
 '유아동스포츠샌들/슬리퍼': 58,
 '골프패션잡화': 59,
 '염모제': 60,
 '장우산': 61,
 '골프필드용품': 62,
 '텐트': 63,
 '기타에어컨': 64,
 '여성스니커즈': 65,
 '남성정장셔츠': 66,
 '토스터/제빵기

In [57]:
data['clac3_nm'] = data['clac3_nm'].map(lambda x: [clac3_nm_dic[nm] for nm in x])

In [58]:
clac3_matrix = np.zeros((len(data), len(clac3_nm_dic)))
for i in range(len(data)):
    for j in data['clac3_nm'][i]:
        clac3_matrix[i, j] += 1

In [59]:
clac3_matrix

array([[1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [60]:
cols = ['clac3_nm_' + str(i) for i in range(len(clac3_nm_dic))]

In [61]:
clac3_df = pd.DataFrame(clac3_matrix)
clac3_df.columns = cols

In [62]:
clac3_df

,clac3_nm_0,clac3_nm_1,clac3_nm_2,clac3_nm_3,clac3_nm_4,clac3_nm_5,clac3_nm_6,clac3_nm_7,clac3_nm_8,clac3_nm_9,...,clac3_nm_873,clac3_nm_874,clac3_nm_875,clac3_nm_876,clac3_nm_877,clac3_nm_878,clac3_nm_879,clac3_nm_880,clac3_nm_881,clac3_nm_882
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
data_concat = pd.concat([data.iloc[:, :-6], clac1_df, clac2_df, clac3_df, data.iloc[:, -1]], axis=1)

In [64]:
data_concat

,clnt_id,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,...,clac3_nm_874,clac3_nm_875,clac3_nm_876,clac3_nm_877,clac3_nm_878,clac3_nm_879,clac3_nm_880,clac3_nm_881,clac3_nm_882,label
0,0,2,43250,86500,1,2,59,118,922,1844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F20
1,1,9,77777.8,700000,1,9,132.333,1191,1311.11,11800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30
2,6,4,24225,96900,1,4,21.75,87,297.25,1189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F20
3,9,2,10550,21100,1,2,249,498,5049,10098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30
4,12,16,14325.6,229210,1,16,144.938,2319,4187.25,66996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,263094,1,10000,10000,1,1,66,66,513,513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30
149996,263095,2,122000,244000,1,2,220.5,441,1828,3656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30
149997,263096,3,28500,85500,1,3,256,768,4237,12711,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30
149998,263102,1,1080,1080,1,1,188,188,1812,1812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30


In [65]:
data_concat.to_csv('./train_preprocessed.csv', index=False)
data_concat = pd.read_csv('./train_preprocessed.csv')

In [66]:
lookup_table_clac1 = []
for i in range(37):
    # clac1_nm_number 이 1 이상인 것들 라벨 별(6차원) value count => 라벨별 분류 별 개수 파악
    prob = data_concat[data_concat['clac1_nm_' + str(i)] >= 1].groupby('label')['label'].value_counts().values
    if len(prob) != 6:
        print(i, prob)
    lookup_table_clac1.append(prob)

In [67]:
lookup_table_clac1 = np.stack(lookup_table_clac1)

- 가중치 스케일링 진행 

In [68]:
weight = np.array([59892/17727, 59892/59892, 59892/51936, 59892/2588, 59892/7953, 59892/9904])
weight

array([ 3.37857506,  1.        ,  1.15318854, 23.14219474,  7.53074312,
        6.04725363])

In [69]:
lookup_table_clac1 = lookup_table_clac1 * weight

In [70]:
lookup_table_clac1 = lookup_table_clac1 / lookup_table_clac1.sum(1).reshape(-1, 1)

In [71]:
lookup_table_clac1

array([[0.1196019 , 0.15351727, 0.20977359, 0.12166872, 0.17288697,
        0.22255155],
       [0.10090549, 0.22714818, 0.27848079, 0.07186492, 0.14375322,
        0.1778474 ],
       [0.10631361, 0.2387599 , 0.22268107, 0.0900885 , 0.15848887,
        0.18366804],
       [0.29039753, 0.19554586, 0.18492875, 0.11494716, 0.10232984,
        0.11185087],
       [0.12042409, 0.17029651, 0.21638294, 0.08393391, 0.18585456,
        0.223108  ],
       [0.13488386, 0.23263313, 0.22147735, 0.09935707, 0.14373666,
        0.16791193],
       [0.11902816, 0.13211609, 0.16521248, 0.21401495, 0.17611567,
        0.19351265],
       [0.16240187, 0.19808438, 0.20762432, 0.1376104 , 0.14092544,
        0.1533536 ],
       [0.07940145, 0.22461032, 0.27383925, 0.05948637, 0.1576259 ,
        0.2050367 ],
       [0.03879911, 0.36162852, 0.26285742, 0.01136482, 0.1382575 ,
        0.18709264],
       [0.08528045, 0.2160113 , 0.33747026, 0.03723632, 0.11662744,
        0.20737422],
       [0.09358846, 0

- 룩업 테이블 중 6차원이 되지 못하는 것들 결측치 보완해 6차원 만들어주기

In [72]:
lookup_table_clac2 = []
for i in range(128):
    prob = data_concat[data_concat['clac2_nm_' + str(i)] >= 1].groupby('label')['label'].value_counts().values
    if len(prob) != 6:
        print(i, prob)
    lookup_table_clac2.append(prob)

42 [133 607 646  27  64]
52 [  42 1780 1083   87  134]
86 [  18 1216 1003   58  147]
111 [  7 605 370  34  39]
120 [ 21  90 124  21  35]
122 [ 20 759 471  47  63]
127 [ 1  9 10  4  4]


In [73]:
data_concat[data_concat['clac2_nm_' + str(86)] >= 1].groupby('label')['label'].value_counts()

label  label
F20    F20        18
F30    F30      1216
F40    F40      1003
M30    M30        58
M40    M40       147
Name: label, dtype: int64

In [74]:
lookup_table_clac2[13] = np.array([56, 1238, 504, 0, 58, 71])
lookup_table_clac2[42] = np.array([109, 519, 550, 0, 21, 52])
lookup_table_clac2[52] = np.array([23, 1036, 685, 0, 52, 75])
lookup_table_clac2[86] = np.array([9, 724, 644, 0, 39, 90])
lookup_table_clac2[111] = np.array([5, 420, 261, 0, 28, 29])
lookup_table_clac2[120] = np.array([18, 88, 121, 0, 20, 35])
lookup_table_clac2[122] = np.array([18, 535, 349, 0, 37, 44])
lookup_table_clac2[127] = np.array([1, 9, 10, 0, 4, 4])

In [75]:
lookup_table_clac2 = np.stack(lookup_table_clac2)

In [76]:
lookup_table_clac2

array([[ 167,  791, 1075,   19,  100,  193],
       [  71,  454,  706,    7,   45,   86],
       [ 776, 5888, 4762,   96,  519,  749],
       [2576, 8291, 7464,  175,  595,  777],
       [ 347, 1245, 1276,   18,   86,  152],
       [ 324, 1687, 1673,   36,  257,  343],
       [5173, 9270, 6888,  160,  339,  613],
       [1158, 4240, 4284,   28,  160,  401],
       [ 741, 2125, 2624,  134,  373,  439],
       [  75, 3239, 1694,    3,  149,  233],
       [  59, 1537,  578,    2,   64,   92],
       [1046, 4946, 5185,   47,  276,  482],
       [  96,  887, 1001,   10,   78,  143],
       [  56, 1238,  504,    0,   58,   71],
       [ 181, 1942, 1187,   25,  169,  182],
       [  72,  697, 1094,    4,   55,  134],
       [  89,  535,  702,   15,   80,  124],
       [ 103,  800,  845,    8,   78,  125],
       [ 131, 1686,  846,    4,  131,  122],
       [  18,   97,  162,    2,   16,   47],
       [ 158,  654,  554,   11,   54,   69],
       [ 755, 2113, 2934,  400,  772,  842],
       [ 1

In [77]:
lookup_table_clac2 = lookup_table_clac2 * weight

In [78]:
lookup_table_clac2 = lookup_table_clac2 / lookup_table_clac2.sum(1).reshape(-1, 1)

In [79]:
lookup_table_clac2

array([[0.11387393, 0.15964331, 0.25019754, 0.08874265, 0.15198897,
        0.2355536 ],
       [0.09485229, 0.17951954, 0.32192959, 0.0640558 , 0.13400044,
        0.20564233],
       [0.10631361, 0.2387599 , 0.22268107, 0.0900885 , 0.15848887,
        0.18366804],
       [0.22413044, 0.21351497, 0.22166308, 0.10429512, 0.11539214,
        0.12100425],
       [0.19964605, 0.21201521, 0.25058131, 0.07093731, 0.11028945,
        0.15653067],
       [0.11457987, 0.17658134, 0.20194169, 0.08720407, 0.20258192,
        0.2171111 ],
       [0.39140263, 0.20760004, 0.17788575, 0.08292247, 0.05717224,
        0.08301687],
       [0.22523187, 0.24409201, 0.28440517, 0.03730356, 0.06936582,
        0.13960156],
       [0.15435505, 0.1310171 , 0.1865663 , 0.19119582, 0.17318717,
        0.16367855],
       [0.03149144, 0.40253967, 0.24277919, 0.00862827, 0.13945107,
        0.17511036],
       [0.05715761, 0.4407196 , 0.19112463, 0.01327159, 0.13819945,
        0.15952712],
       [0.1720522 , 0

In [80]:
lookup_table_clac3 = np.zeros((883, 6))
# 888*6의 zero nd array를 생성 initialize

for i in range(883):
    try:
        prob = data_concat[data_concat['clac3_nm_' + str(i)] >= 1].groupby('label')['label'].value_counts()
        for j in range(len(prob.keys())):
            if prob.keys()[j][0] == 'F20':
                lookup_table_clac3[i, 0] = prob.values[j]
            elif prob.keys()[j][0] == 'F30':
                lookup_table_clac3[i, 1] = prob.values[j]
            elif prob.keys()[j][0] == 'F40':
                lookup_table_clac3[i, 2] = prob.values[j]
            elif prob.keys()[j][0] == 'M20':
                lookup_table_clac3[i, 3] = prob.values[j]
            elif prob.keys()[j][0] == 'M30':
                lookup_table_clac3[i, 4] = prob.values[j]
            elif prob.keys()[j][0] == 'M40':
                lookup_table_clac3[i, 5] = prob.values[j]
    except:
        lookup_table_clac3[i] = 1/weight

In [81]:
lookup_table_clac3

array([[3.800e+01, 1.350e+02, 2.320e+02, 3.000e+00, 1.700e+01, 3.300e+01],
       [2.400e+01, 2.210e+02, 2.980e+02, 0.000e+00, 1.800e+01, 3.900e+01],
       [1.620e+02, 1.093e+03, 1.021e+03, 4.000e+01, 1.040e+02, 1.860e+02],
       ...,
       [0.000e+00, 1.000e+00, 2.000e+00, 0.000e+00, 0.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00],
       [0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00]])

In [82]:
lookup_table_clac3 = lookup_table_clac3 * weight

In [83]:
lookup_table_clac3 = lookup_table_clac3 / lookup_table_clac3.sum(1).reshape(-1, 1)

In [84]:
lookup_table_clac3

array([[0.13835664, 0.14548446, 0.28831758, 0.07481844, 0.13796521,
        0.21505768],
       [0.07972002, 0.21727755, 0.33786185, 0.        , 0.13327016,
        0.23187043],
       [0.09684827, 0.1934031 , 0.2083384 , 0.1637977 , 0.13858443,
        0.1990281 ],
       ...,
       [0.        , 0.30244584, 0.69755416, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ]])

In [85]:
mat_clac1 = np.array(data_concat.iloc[:, 11:48]).dot(lookup_table_clac1)
mat_clac1 = mat_clac1 / mat_clac1.sum(1).reshape(-1, 1)

In [86]:
mat_clac1 = pd.DataFrame(mat_clac1)
mat_clac1 = mat_clac1.rename(columns={0: 'given_clac1_F20_prob', 1:'given_clac1_F30_prob', 2:'given_clac1_F40_prob', 3:'given_clac1_M20_prob', 4:'given_clac1_M30_prob', 5:'given_clac1_M40_prob'})

- 본격 룩업 테이블에서 벡터 뽑아내는 코드

In [87]:
mat_buy_clac1 = np.where(np.array(data_concat.iloc[:, 11:48]) >= 1, 1, 0).dot(lookup_table_clac1)
mat_buy_clac1 = mat_buy_clac1 / mat_buy_clac1.sum(1).reshape(-1, 1)

In [88]:
mat_buy_clac1 = pd.DataFrame(mat_buy_clac1)
mat_buy_clac1 = mat_buy_clac1.rename(columns={0: 'buy_clac1_F20_prob', 1:'buy_clac1_F30_prob', 2:'buy_clac1_F40_prob', 3:'buy_clac1_M20_prob', 4:'buy_clac1_M30_prob', 5:'buy_clac1_M40_prob'})

In [89]:
mat_clac2 = np.array(data_concat.iloc[:, 48:-884]).dot(lookup_table_clac2)
mat_clac2 = mat_clac2 / mat_clac2.sum(1).reshape(-1, 1)

In [90]:
mat_clac2 = pd.DataFrame(mat_clac2)
mat_clac2 = mat_clac2.rename(columns={0: 'given_clac2_F20_prob', 1:'given_clac2_F30_prob', 2:'given_clac2_F40_prob', 3:'given_clac2_M20_prob', 4:'given_clac2_M30_prob', 5:'given_clac2_M40_prob'})

In [91]:
mat_buy_clac2 = np.where(np.array(data_concat.iloc[:, 48:-884]) >= 1, 1, 0).dot(lookup_table_clac2)
mat_buy_clac2 = mat_buy_clac2 / mat_buy_clac2.sum(1).reshape(-1, 1)

In [92]:
mat_buy_clac2 = pd.DataFrame(mat_buy_clac2)
mat_buy_clac2 = mat_buy_clac2.rename(columns={0: 'buy_clac2_F20_prob', 1:'buy_clac2_F30_prob', 2:'buy_clac2_F40_prob', 3:'buy_clac2_M20_prob', 4:'buy_clac2_M30_prob', 5:'buy_clac2_M40_prob'})

In [93]:
mat_clac3 = np.array(data_concat.iloc[:, -884:-1]).dot(lookup_table_clac3)
mat_clac3 = mat_clac3 / mat_clac3.sum(1).reshape(-1, 1)

In [94]:
mat_clac3 = pd.DataFrame(mat_clac3)
mat_clac3 = mat_clac3.rename(columns={0: 'given_clac3_F20_prob', 1:'given_clac3_F30_prob', 2:'given_clac3_F40_prob', 3:'given_clac3_M20_prob', 4:'given_clac3_M30_prob', 5:'given_clac3_M40_prob'})

In [95]:
mat_buy_clac3 = np.where(np.array(data_concat.iloc[:, -884:-1]) >= 1, 1, 0).dot(lookup_table_clac3)
mat_buy_clac3 = mat_buy_clac3 / mat_buy_clac3.sum(1).reshape(-1, 1)

In [96]:
mat_buy_clac3 = pd.DataFrame(mat_buy_clac3)
mat_buy_clac3 = mat_buy_clac3.rename(columns={0: 'buy_clac3_F20_prob', 1:'buy_clac3_F30_prob', 2:'buy_clac3_F40_prob', 3:'buy_clac3_M20_prob', 4:'buy_clac3_M30_prob', 5:'buy_clac3_M40_prob'})

clac1_embed = np.load('C:/Users/user/Desktop/DHCN-main/datasets/clac1_nm/embedding.npy')

train_clac1_mean = np.array(data_concat.iloc[:, 11:48]).dot(clac1_embed)
train_clac1_mean = train_clac1_mean / train_clac1_mean.sum(1).reshape(-1, 1)

train_clac1_weight = np.where(np.array(data_concat.iloc[:, 11:48]) >= 1, 1, 0).dot(clac1_embed)
train_clac1_weight = train_clac1_weight / train_clac1_weight.sum(1).reshape(-1, 1)

train_clac1_mean = pd.DataFrame(train_clac1_mean)
train_clac1_weight = pd.DataFrame(train_clac1_weight)
train_clac1_mean.columns = ['clac1_mean_'+str(i) for i in range(5)]
train_clac1_weight.columns = ['clac1_weight_'+str(i) for i in range(5)]

clac2_embed = np.load('C:/Users/user/Desktop/DHCN-main/datasets/clac2_nm/embedding.npy')

train_clac2_mean = np.array(data_concat.iloc[:, 48:-884]).dot(clac2_embed)
train_clac2_mean = train_clac2_mean / train_clac2_mean.sum(1).reshape(-1, 1)

train_clac2_weight = np.where(np.array(data_concat.iloc[:, 48:-884]) >= 1, 1, 0).dot(clac2_embed)
train_clac2_weight = train_clac2_weight / train_clac2_weight.sum(1).reshape(-1, 1)

train_clac2_mean = pd.DataFrame(train_clac2_mean)
train_clac2_weight = pd.DataFrame(train_clac2_weight)
train_clac2_mean.columns = ['clac2_mean_'+str(i) for i in range(10)]
train_clac2_weight.columns = ['clac2_weight_'+str(i) for i in range(10)]

clac3_embed = np.load('C:/Users/user/Desktop/DHCN-main/datasets/clac3_nm/embedding.npy')

train_clac3_mean = np.array(data_concat.iloc[:, -884:-1]).dot(clac3_embed[:883])
train_clac3_mean = train_clac3_mean / train_clac3_mean.sum(1).reshape(-1, 1)

train_clac3_weight = np.where(np.array(data_concat.iloc[:, -884:-1]) >= 1, 1, 0).dot(clac3_embed[:883])
train_clac3_weight = train_clac3_weight / train_clac3_weight.sum(1).reshape(-1, 1)

train_clac3_mean = pd.DataFrame(train_clac3_mean)
train_clac3_weight = pd.DataFrame(train_clac3_weight)
train_clac3_mean.columns = ['clac3_mean_'+str(i) for i in range(30)]
train_clac3_weight.columns = ['clac3_weight_'+str(i) for i in range(30)]

In [97]:
data_concat['gender'] = data_concat['label'].map(lambda x: x[:1])
data_concat['age'] = data_concat['label'].map(lambda x: x[1:])

In [98]:
data_concat

,clnt_id,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,...,clac3_nm_876,clac3_nm_877,clac3_nm_878,clac3_nm_879,clac3_nm_880,clac3_nm_881,clac3_nm_882,label,gender,age
0,0,2,43250.000000,86500,1.0,2,59.000000,118.0,922.000000,1844,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F20,F,20
1,1,9,77777.777778,700000,1.0,9,132.333333,1191.0,1311.111111,11800,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30
2,6,4,24225.000000,96900,1.0,4,21.750000,87.0,297.250000,1189,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F20,F,20
3,9,2,10550.000000,21100,1.0,2,249.000000,498.0,5049.000000,10098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30
4,12,16,14325.625000,229210,1.0,16,144.937500,2319.0,4187.250000,66996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,263094,1,10000.000000,10000,1.0,1,66.000000,66.0,513.000000,513,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30
149996,263095,2,122000.000000,244000,1.0,2,220.500000,441.0,1828.000000,3656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30
149997,263096,3,28500.000000,85500,1.0,3,256.000000,768.0,4237.000000,12711,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30
149998,263102,1,1080.000000,1080,1.0,1,188.000000,188.0,1812.000000,1812,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,F30,F,30


In [99]:
label_dic = {}
for i, label in enumerate(['F20', 'F30', 'F40', 'M20', 'M30', 'M40']):
    label_dic[label] = i

In [100]:
label_dic

{'F20': 0, 'F30': 1, 'F40': 2, 'M20': 3, 'M30': 4, 'M40': 5}

In [101]:
gender_dic = {}
for i, gender in enumerate(['F', 'M']):
    gender_dic[gender] = i

In [102]:
gender_dic

{'F': 0, 'M': 1}

In [103]:
age_dic = {}
for i, age in enumerate(['20', '30', '40']):
    age_dic[age] = i

In [104]:
age_dic

{'20': 0, '30': 1, '40': 2}

In [105]:
data_concat['label'] = data_concat['label'].map(lambda x: label_dic[x])
data_concat['gender'] = data_concat['gender'].map(lambda x: gender_dic[x])
data_concat['age'] = data_concat['age'].map(lambda x: age_dic[x])

In [106]:
(np.array(mat_clac1).argmax(1) == data_concat['label'].values).sum()  # 50456

52069

In [107]:
(np.array(mat_buy_clac1).argmax(1) == data_concat['label'].values).sum()  # 51031

52940

In [108]:
(np.array(mat_clac2).argmax(1) == data_concat['label'].values).sum()  # 55530

57561

In [109]:
(np.array(mat_buy_clac2).argmax(1) == data_concat['label'].values).sum()  # 55932

58263

In [110]:
(np.array(mat_clac3).argmax(1) == data_concat['label'].values).sum()  # 56845

58976

In [111]:
(np.array(mat_buy_clac3).argmax(1) == data_concat['label'].values).sum()  # 57160

59451

### word2vec

In [112]:
from gensim.models import word2vec

In [113]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
        lst.append(tmp)
    return lst

#### Product code Word2vec

In [114]:
df['PD_C'] = df['PD_C'].astype('string')
df_test['PD_C'] = df_test['PD_C'].astype('string')

In [115]:
train_pd_w2v = list(df.groupby('CLNT_ID')['PD_C'].unique()[y['CLNT_ID'].values])
test_pd_w2v = list(df_test.groupby('CLNT_ID')['PD_C'].unique()[df_test['CLNT_ID'].unique()])

In [116]:
tt = df.groupby('CLNT_ID')['PD_C'].value_counts()[y['CLNT_ID'].values]
kk = y['CLNT_ID'].values
ww = df.groupby(['CLNT_ID', 'PD_C'])['KWD_NM'].count()

In [117]:
train_pd_w2v.extend(test_pd_w2v)

In [118]:
pd_w2v_input = oversample(train_pd_w2v, 10)

In [119]:
# product code word2vec
pd_w2v = word2vec.Word2Vec(sentences=pd_w2v_input,
                           size=256,
                           window=3,
                           min_count=1,
                           sg=1).wv

In [120]:
pd_mean_vector = []

for words in tqdm(train_pd_w2v[:-len(test_pd_w2v)]):
        tmp = np.zeros(256) 
        cnt = 0
        for word in words:
            try:
                tmp += pd_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        pd_mean_vector.append(tmp)
        
pd_mean_vector = np.array(pd_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:05<00:00, 28873.49it/s]


In [121]:
pd_unweight_vector = []

for i, words in tqdm(enumerate(train_pd_w2v[:-len(test_pd_w2v)])):
        tmp = np.zeros(256) 
        cnt = 0
        for word in words:
            try:
                tmp += pd_w2v[word] * tt[kk[i]][word]
                cnt += tt[kk[i]][word]
            except:
                pass
        #tmp /= cnt 
        pd_unweight_vector.append(tmp)
        
pd_unweight_vector = np.array(pd_unweight_vector)

150000it [04:30, 553.55it/s]


In [122]:
pd_weight_vector = []

for i, words in tqdm(enumerate(train_pd_w2v[:-len(test_pd_w2v)])):
        tmp = np.zeros(256) 
        cnt = 0
        for word in words:
            try:
                tmp += pd_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        pd_weight_vector.append(tmp)
        
pd_weight_vector = np.array(pd_weight_vector)

150000it [12:04, 207.08it/s]


In [123]:
train_pd_mean = pd_mean_vector
train_pd_unweight = pd_unweight_vector
train_pd_weight = pd_weight_vector

train_pd_mean = pd.DataFrame(train_pd_mean)
train_pd_unweight = pd.DataFrame(train_pd_unweight)
train_pd_weight = pd.DataFrame(train_pd_weight)

train_pd_mean.columns = ['pd_mean_'+str(i) for i in range(256)]
train_pd_unweight.columns = ['pd_unweight_'+str(i) for i in range(256)]
train_pd_weight.columns = ['pd_weight_'+str(i) for i in range(256)]

In [124]:
df['PD_BRA_NM'] = df['PD_BRA_NM'].astype('string')
df_test['PD_BRA_NM'] = df_test['PD_BRA_NM'].astype('string')

#### brand name word2vec

In [125]:
train_bra_w2v = list(df.groupby('CLNT_ID')['PD_BRA_NM'].unique()[y['CLNT_ID'].values])
test_bra_w2v = list(df_test.groupby('CLNT_ID')['PD_BRA_NM'].unique()[df_test['CLNT_ID'].unique()])

In [126]:
tt = df.groupby('CLNT_ID')['PD_BRA_NM'].value_counts()[y['CLNT_ID'].values]
## tt : kk에 해당하는 product name count 값
kk = y['CLNT_ID'].values
## kk : client id list
ww = df.groupby(['CLNT_ID', 'PD_BRA_NM'])['KWD_NM'].count()
## client id, brand name로 그룹핑한 keyword들마다의 개수

In [127]:
train_bra_w2v.extend(test_bra_w2v)

In [128]:
bra_w2v_input = oversample(train_bra_w2v, 10)

In [129]:
bra_w2v = word2vec.Word2Vec(sentences=bra_w2v_input,
                           size=128,
                           window=3,
                           min_count=1,
                           sg=1).wv

In [130]:
bra_mean_vector = []

for words in tqdm(train_bra_w2v[:-len(test_bra_w2v)]):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += bra_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        bra_mean_vector.append(tmp)
        
bra_mean_vector = np.array(bra_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:03<00:00, 40477.85it/s]


In [131]:
bra_unweight_vector = []

for i, words in tqdm(enumerate(train_bra_w2v[:-len(test_bra_w2v)])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += bra_w2v[word] * tt[kk[i]][word]
                cnt += tt[kk[i]][word]
            except:
                pass
        #tmp /= cnt 
        bra_unweight_vector.append(tmp)
        
bra_unweight_vector = np.array(bra_unweight_vector)

150000it [03:17, 758.58it/s]


In [132]:
bra_weight_vector = []

for i, words in tqdm(enumerate(train_bra_w2v[:-len(test_bra_w2v)])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += bra_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        bra_weight_vector.append(tmp)
        
bra_weight_vector = np.array(bra_weight_vector)

150000it [08:57, 278.88it/s]


In [133]:
train_bra_mean = bra_mean_vector
train_bra_unweight = bra_unweight_vector
train_bra_weight = bra_weight_vector

train_bra_mean = pd.DataFrame(train_bra_mean)
train_bra_unweight = pd.DataFrame(train_bra_unweight)
train_bra_weight = pd.DataFrame(train_bra_weight)

train_bra_mean.columns = ['bra_mean_'+str(i) for i in range(128)]
train_bra_unweight.columns = ['bra_unweight_'+str(i) for i in range(128)]
train_bra_weight.columns = ['bra_weight_'+str(i) for i in range(128)]

#### keyword_name word2vec

In [134]:
df['KWD_NM'] = df['KWD_NM'].astype('string')
df_test['KWD_NM'] = df_test['KWD_NM'].astype('string')

In [135]:
train_kwd_w2v = list(df.groupby('CLNT_ID')['KWD_NM'].unique()[y['CLNT_ID'].values])
test_kwd_w2v = list(df_test.groupby('CLNT_ID')['KWD_NM'].unique()[df_test['CLNT_ID'].unique()])

In [136]:
tt = df.groupby('CLNT_ID')['KWD_NM'].value_counts()[y['CLNT_ID'].values]
kk = y['CLNT_ID'].values
ww = df.groupby(['CLNT_ID', 'KWD_NM'])['KWD_NM'].count()

In [137]:
train_kwd_w2v.extend(test_kwd_w2v)

In [138]:
kwd_w2v_input = oversample(train_kwd_w2v, 10)

In [139]:
kwd_w2v = word2vec.Word2Vec(sentences=kwd_w2v_input,
                           size=128,
                           window=3,
                           min_count=1,
                           sg=1).wv

In [140]:
kwd_mean_vector = []

for words in tqdm(train_kwd_w2v[:-len(test_kwd_w2v)]):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += kwd_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        kwd_mean_vector.append(tmp)
        
kwd_mean_vector = np.array(kwd_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:06<00:00, 23661.92it/s]


In [141]:
kwd_unweight_vector = []

for i, words in tqdm(enumerate(train_kwd_w2v[:-len(test_kwd_w2v)])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += kwd_w2v[word] * tt[kk[i]][word]
                cnt += tt[kk[i]][word]
            except:
                pass
        #tmp /= cnt 
        kwd_unweight_vector.append(tmp)
        
kwd_unweight_vector = np.array(kwd_unweight_vector)

150000it [05:42, 437.61it/s]


In [142]:
kwd_weight_vector = []

for i, words in tqdm(enumerate(train_kwd_w2v[:-len(test_kwd_w2v)])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += kwd_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        kwd_weight_vector.append(tmp)
        
kwd_weight_vector = np.array(kwd_weight_vector)

150000it [15:38, 159.76it/s]


In [143]:
train_kwd_mean = kwd_mean_vector
train_kwd_unweight = kwd_unweight_vector
train_kwd_weight = kwd_weight_vector

train_kwd_mean = pd.DataFrame(train_kwd_mean)
train_kwd_unweight = pd.DataFrame(train_kwd_unweight)
train_kwd_weight = pd.DataFrame(train_kwd_weight)

train_kwd_mean.columns = ['kwd_mean_'+str(i) for i in range(128)]
train_kwd_weight.columns = ['kwd_unweight_'+str(i) for i in range(128)]
train_kwd_weight.columns = ['kwd_weight_'+str(i) for i in range(128)]

#### CLAC3_NM word2vec

In [144]:
df['CLAC3_NM'] = df['CLAC3_NM'].astype('string')
df_test['CLAC3_NM'] = df_test['CLAC3_NM'].astype('string')

In [145]:
train_clac3_w2v = list(df.groupby('CLNT_ID')['CLAC3_NM'].unique()[y['CLNT_ID'].values])
test_clac3_w2v = list(df_test.groupby('CLNT_ID')['CLAC3_NM'].unique()[df_test['CLNT_ID'].unique()])

In [146]:
tt = df.groupby('CLNT_ID')['CLAC3_NM'].value_counts()[y['CLNT_ID'].values]
kk = y['CLNT_ID'].values
ww = df.groupby(['CLNT_ID', 'CLAC3_NM'])['KWD_NM'].count()

In [147]:
train_clac3_w2v.extend(test_clac3_w2v)

In [148]:
clac3_w2v_input = oversample(train_clac3_w2v, 5)

In [149]:
clac3_w2v = word2vec.Word2Vec(sentences=clac3_w2v_input,
                              size=30,
                              window=3,
                              min_count=1,
                              sg=1).wv

In [150]:
clac3_mean_vector = []

for words in tqdm(train_clac3_w2v[:-len(test_clac3_w2v)]):
        tmp = np.zeros(30) 
        cnt = 0
        for word in words:
            try:
                tmp += clac3_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        clac3_mean_vector.append(tmp)
        
clac3_mean_vector = np.array(clac3_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:03<00:00, 40631.44it/s]


In [151]:
clac3_unweight_vector = []

for i, words in tqdm(enumerate(train_clac3_w2v[:-len(test_clac3_w2v)])):
        tmp = np.zeros(30) 
        cnt = 0
        for word in words:
            try:
                tmp += clac3_w2v[word] * clac3_matrix[i, clac3_nm_dic[word]]
                cnt += clac3_matrix[i, clac3_nm_dic[word]]
            except:
                pass
        #tmp /= cnt 
        clac3_unweight_vector.append(tmp)
        
clac3_unweight_vector = np.array(clac3_unweight_vector)

150000it [00:05, 26368.42it/s]


In [152]:
clac3_weight_vector = []

for i, words in tqdm(enumerate(train_clac3_w2v[:-len(test_clac3_w2v)])):
        tmp = np.zeros(30) 
        cnt = 0
        for word in words:
            try:
                tmp += clac3_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        clac3_weight_vector.append(tmp)
        
clac3_weight_vector = np.array(clac3_weight_vector)

150000it [08:58, 278.79it/s]


In [153]:
train_clac3_mean = clac3_mean_vector
train_clac3_unweight = clac3_unweight_vector
train_clac3_weight = clac3_weight_vector

train_clac3_mean = pd.DataFrame(train_clac3_mean)
train_clac3_unweight = pd.DataFrame(train_clac3_unweight)
train_clac3_weight = pd.DataFrame(train_clac3_weight)

train_clac3_mean.columns = ['clac3_mean_'+str(i) for i in range(30)]
train_clac3_unweight.columns = ['clac3_unweight_'+str(i) for i in range(30)]
train_clac3_weight.columns = ['clac3_weight_'+str(i) for i in range(30)]

#### CLAC2_NM word2vec

In [154]:
df['CLAC2_NM'] = df['CLAC2_NM'].astype('string')
df_test['CLAC2_NM'] = df_test['CLAC2_NM'].astype('string')

In [155]:
train_clac2_w2v = list(df.groupby('CLNT_ID')['CLAC2_NM'].unique()[y['CLNT_ID'].values])
test_clac2_w2v = list(df_test.groupby('CLNT_ID')['CLAC2_NM'].unique()[df_test['CLNT_ID'].unique()])

In [156]:
tt = df.groupby('CLNT_ID')['CLAC2_NM'].value_counts()[y['CLNT_ID'].values]
kk = y['CLNT_ID'].values
ww = df.groupby(['CLNT_ID', 'CLAC2_NM'])['KWD_NM'].count()

In [157]:
train_clac2_w2v.extend(test_clac2_w2v)

In [158]:
clac2_w2v_input = oversample(train_clac2_w2v, 5)

In [159]:
clac2_w2v = word2vec.Word2Vec(sentences=clac2_w2v_input,
                              size=10,
                              window=3,
                              min_count=1,
                              sg=1).wv

In [160]:
clac2_mean_vector = []

for words in tqdm(train_clac2_w2v[:-len(test_clac2_w2v)]):
        tmp = np.zeros(10) 
        cnt = 0
        for word in words:
            try:
                tmp += clac2_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt 
        clac2_mean_vector.append(tmp)
        
clac2_mean_vector = np.array(clac2_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:03<00:00, 45694.59it/s]


In [161]:
clac2_unweight_vector = []

for i, words in tqdm(enumerate(train_clac2_w2v[:-len(test_clac2_w2v)])):
        tmp = np.zeros(10) 
        cnt = 0
        for word in words:
            try:
                tmp += clac2_w2v[word] * clac2_matrix[i, clac2_nm_dic[word]]
                cnt += clac2_matrix[i, clac2_nm_dic[word]]
            except:
                pass
        #tmp /= cnt 
        clac2_unweight_vector.append(tmp)
        
clac2_unweight_vector = np.array(clac2_unweight_vector)

150000it [00:04, 31135.69it/s]


In [162]:
clac2_weight_vector = []

for i, words in tqdm(enumerate(train_clac2_w2v[:-len(test_clac2_w2v)])):
        tmp = np.zeros(10) 
        cnt = 0
        for word in words:
            try:
                tmp += clac2_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        clac2_weight_vector.append(tmp)
        
clac2_weight_vector = np.array(clac2_weight_vector)

150000it [07:40, 325.76it/s]


In [163]:
train_clac2_mean = clac2_mean_vector
train_clac2_unweight = clac2_unweight_vector
train_clac2_weight = clac2_weight_vector

train_clac2_mean = pd.DataFrame(train_clac2_mean)
train_clac2_unweight = pd.DataFrame(train_clac2_unweight)
train_clac2_weight = pd.DataFrame(train_clac2_weight)

train_clac2_mean.columns = ['clac2_mean_'+str(i) for i in range(10)]
train_clac2_unweight.columns = ['clac2_unweight_'+str(i) for i in range(10)]
train_clac2_weight.columns = ['clac2_weight_'+str(i) for i in range(10)]

#### CLAC1_NM word2vec

In [164]:
df['CLAC1_NM'] = df['CLAC1_NM'].astype('string')
df_test['CLAC1_NM'] = df_test['CLAC1_NM'].astype('string')

In [165]:
train_clac1_w2v = list(df.groupby('CLNT_ID')['CLAC1_NM'].unique()[y['CLNT_ID'].values])
test_clac1_w2v = list(df_test.groupby('CLNT_ID')['CLAC1_NM'].unique()[df_test['CLNT_ID'].unique()])

In [166]:
tt = df.groupby('CLNT_ID')['CLAC1_NM'].value_counts()[y['CLNT_ID'].values]
kk = y['CLNT_ID'].values
ww = df.groupby(['CLNT_ID', 'CLAC1_NM'])['KWD_NM'].count()

In [167]:
train_clac1_w2v.extend(test_clac1_w2v)

In [168]:
clac1_w2v_input = oversample(train_clac1_w2v, 5)

In [169]:
clac1_w2v = word2vec.Word2Vec(sentences=clac1_w2v_input,
                              size=5,
                              window=3,
                              min_count=1,
                              sg=1).wv

In [170]:
clac1_mean_vector = []

for words in tqdm(train_clac1_w2v[:-len(test_clac1_w2v)]):
        tmp = np.zeros(5) 
        cnt = 0
        for word in words:
            try:
                tmp += clac1_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt 
        clac1_mean_vector.append(tmp)
        
clac1_mean_vector = np.array(clac1_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 150000/150000 [00:02<00:00, 56621.56it/s]


In [171]:
clac1_unweight_vector = []

for i, words in tqdm(enumerate(train_clac1_w2v[:-len(test_clac1_w2v)])):
        tmp = np.zeros(5) 
        cnt = 0
        for word in words:
            try:
                tmp += clac1_w2v[word] * clac1_matrix[i, clac1_nm_dic[word]]
                cnt += clac1_matrix[i, clac1_nm_dic[word]]
            except:
                pass
        #tmp /= cnt 
        clac1_unweight_vector.append(tmp)
        
clac1_unweight_vector = np.array(clac1_unweight_vector)

150000it [00:03, 38266.09it/s]


In [174]:
clac1_weight_vector = []

for i, words in tqdm(enumerate(train_clac1_w2v[:-len(test_clac1_w2v)])):
        tmp = np.zeros(5) 
        cnt = 0
        for word in words:
            try:
                tmp += clac1_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        clac1_weight_vector.append(tmp)
        
clac1_weight_vector = np.array(clac1_weight_vector)

150000it [05:59, 416.81it/s]


In [175]:
train_clac1_mean = clac1_mean_vector
train_clac1_unweight = clac1_unweight_vector
train_clac1_weight = clac1_weight_vector

train_clac1_mean = pd.DataFrame(train_clac1_mean)
train_clac1_unweight = pd.DataFrame(train_clac1_unweight)
train_clac1_weight = pd.DataFrame(train_clac1_weight)

train_clac1_mean.columns = ['clac1_mean_'+str(i) for i in range(5)]
train_clac1_unweight.columns = ['clac1_unweight_'+str(i) for i in range(5)]
train_clac1_weight.columns = ['clac1_weight_'+str(i) for i in range(5)]

In [176]:
data_concat = pd.concat([data_concat.iloc[:, 1:11], mat_clac1, mat_buy_clac1, mat_clac2, mat_buy_clac2, mat_clac3, mat_buy_clac3, train_pd_mean, train_bra_mean, train_kwd_mean, train_clac1_mean, train_clac2_mean, train_clac3_mean, train_pd_unweight, train_bra_unweight, train_kwd_unweight, train_clac1_unweight, train_clac2_unweight, train_clac3_unweight, train_pd_weight, train_bra_weight, train_kwd_weight, train_clac1_weight, train_clac2_weight, train_clac3_weight, data_concat.iloc[:, -3:]], axis=1)

In [177]:
data_concat

,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,avg_shopping_interval,...,clac3_weight_23,clac3_weight_24,clac3_weight_25,clac3_weight_26,clac3_weight_27,clac3_weight_28,clac3_weight_29,label,gender,age
0,2,43250.000000,86500,1.0,2,59.000000,118.0,922.000000,1844,0.000000,...,-0.025476,-0.358678,-0.186819,0.192818,0.434122,-0.107545,0.063859,0,0,0
1,9,77777.777778,700000,1.0,9,132.333333,1191.0,1311.111111,11800,3.625000,...,-0.513720,0.286228,0.180560,0.383307,0.273483,0.445723,-0.301021,1,0,1
2,4,24225.000000,96900,1.0,4,21.750000,87.0,297.250000,1189,5.666667,...,-0.164428,0.047147,0.298240,-0.305354,0.558061,0.195794,0.031537,0,0,0
3,2,10550.000000,21100,1.0,2,249.000000,498.0,5049.000000,10098,0.000000,...,-0.527290,-0.260950,0.230153,-0.477711,-0.060666,-0.118800,-0.559288,1,0,1
4,16,14325.625000,229210,1.0,16,144.937500,2319.0,4187.250000,66996,4.200000,...,-0.257740,-0.554550,-0.216515,-0.312947,-0.201764,0.072622,-0.378338,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1,10000.000000,10000,1.0,1,66.000000,66.0,513.000000,513,183.000000,...,-0.105944,-1.130829,0.387124,0.286016,0.126334,0.426996,0.029120,1,0,1
149996,2,122000.000000,244000,1.0,2,220.500000,441.0,1828.000000,3656,83.000000,...,-0.719438,-0.272537,-0.384826,-0.215291,-0.045238,-0.005138,-0.127866,1,0,1
149997,3,28500.000000,85500,1.0,3,256.000000,768.0,4237.000000,12711,0.000000,...,-0.675275,-0.445863,-0.125119,0.457581,0.359894,0.512207,-0.033372,1,0,1
149998,1,1080.000000,1080,1.0,1,188.000000,188.0,1812.000000,1812,183.000000,...,0.113508,0.063089,0.235095,0.160695,0.016483,0.776254,-0.536355,1,0,1


In [178]:
data_concat = data_concat.loc[:, list(data_concat.columns[:-3].values) + ['gender', 'age', 'label']]

In [179]:
data_concat

,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,avg_shopping_interval,...,clac3_weight_23,clac3_weight_24,clac3_weight_25,clac3_weight_26,clac3_weight_27,clac3_weight_28,clac3_weight_29,gender,age,label
0,2,43250.000000,86500,1.0,2,59.000000,118.0,922.000000,1844,0.000000,...,-0.025476,-0.358678,-0.186819,0.192818,0.434122,-0.107545,0.063859,0,0,0
1,9,77777.777778,700000,1.0,9,132.333333,1191.0,1311.111111,11800,3.625000,...,-0.513720,0.286228,0.180560,0.383307,0.273483,0.445723,-0.301021,0,1,1
2,4,24225.000000,96900,1.0,4,21.750000,87.0,297.250000,1189,5.666667,...,-0.164428,0.047147,0.298240,-0.305354,0.558061,0.195794,0.031537,0,0,0
3,2,10550.000000,21100,1.0,2,249.000000,498.0,5049.000000,10098,0.000000,...,-0.527290,-0.260950,0.230153,-0.477711,-0.060666,-0.118800,-0.559288,0,1,1
4,16,14325.625000,229210,1.0,16,144.937500,2319.0,4187.250000,66996,4.200000,...,-0.257740,-0.554550,-0.216515,-0.312947,-0.201764,0.072622,-0.378338,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1,10000.000000,10000,1.0,1,66.000000,66.0,513.000000,513,183.000000,...,-0.105944,-1.130829,0.387124,0.286016,0.126334,0.426996,0.029120,0,1,1
149996,2,122000.000000,244000,1.0,2,220.500000,441.0,1828.000000,3656,83.000000,...,-0.719438,-0.272537,-0.384826,-0.215291,-0.045238,-0.005138,-0.127866,0,1,1
149997,3,28500.000000,85500,1.0,3,256.000000,768.0,4237.000000,12711,0.000000,...,-0.675275,-0.445863,-0.125119,0.457581,0.359894,0.512207,-0.033372,0,1,1
149998,1,1080.000000,1080,1.0,1,188.000000,188.0,1812.000000,1812,183.000000,...,0.113508,0.063089,0.235095,0.160695,0.016483,0.776254,-0.536355,0,1,1


In [180]:
data_concat.to_csv('train.csv', index=False)

In [48]:
import pandas as pd
import numpy as np

data_concat = pd.read_csv('train.csv')
train_stat = pd.read_csv('stat_df.csv')

In [56]:
data_concat = pd.concat([train_stat.iloc[:, 1:], data_concat.iloc[:, 10:]], axis=1)

# train

In [57]:
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [58]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [84]:
X_train, X_valid, y_train, y_valid = train_test_split(data_concat.iloc[:, :-1], data_concat.iloc[:, -1], test_size=0.2, stratify=data_concat.iloc[:, -1])

In [85]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.iloc[:, :-2])
X_valid_scaled = scaler.transform(X_valid.iloc[:, :-2])

In [86]:
model1 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model1.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model1.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))  # 1.1643

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 14us/sample - loss: 1.2876 - accuracy: 0.4757 - val_loss: 1.1988 - val_accuracy: 0.5098
Epoch 2/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.2033 - accuracy: 0.5061 - val_loss: 1.1858 - val_accuracy: 0.5216
Epoch 3/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1860 - accuracy: 0.5135 - val_loss: 1.1775 - val_accuracy: 0.5181
Epoch 4/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1728 - accuracy: 0.5189 - val_loss: 1.1708 - val_accuracy: 0.5203
Epoch 5/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1614 - accuracy: 0.5215 - val_loss: 1.1714 - val_accuracy: 0.5197
Epoch 6/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1547 - accuracy: 0.5244 - val_loss: 1.1686 - val_accuracy: 0.5238
Epoch 7/1000
120000/120000 [========

In [87]:
model2 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model2.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model2.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 20us/sample - loss: 1.2883 - accuracy: 0.4768 - val_loss: 1.1937 - val_accuracy: 0.5115
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2037 - accuracy: 0.5047 - val_loss: 1.1856 - val_accuracy: 0.5169
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1870 - accuracy: 0.5116 - val_loss: 1.1714 - val_accuracy: 0.5220
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1753 - accuracy: 0.5175 - val_loss: 1.1733 - val_accuracy: 0.5183
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1671 - accuracy: 0.5186 - val_loss: 1.1650 - val_accuracy: 0.5235
Epoch 6/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1574 - accuracy: 0.5222 - val_loss: 1.1639 - val_accuracy: 0.5215
Epoch 7/1000
120000/120000 [===

In [88]:
model3 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model3.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model3.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 13us/sample - loss: 1.2943 - accuracy: 0.4743 - val_loss: 1.1932 - val_accuracy: 0.5142
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2040 - accuracy: 0.5047 - val_loss: 1.1846 - val_accuracy: 0.5208
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1854 - accuracy: 0.5118 - val_loss: 1.1800 - val_accuracy: 0.5165
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1731 - accuracy: 0.5170 - val_loss: 1.1680 - val_accuracy: 0.5226
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1612 - accuracy: 0.5207 - val_loss: 1.1703 - val_accuracy: 0.5201
Epoch 6/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1582 - accuracy: 0.5248 - val_loss: 1.1647 - val_accuracy: 0.5209
Epoch 7/1000
120000/120000 [===

In [89]:
model4 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model4.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model4.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 13us/sample - loss: 1.2951 - accuracy: 0.4740 - val_loss: 1.1908 - val_accuracy: 0.5107
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2020 - accuracy: 0.5061 - val_loss: 1.1779 - val_accuracy: 0.5170
Epoch 3/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1874 - accuracy: 0.5119 - val_loss: 1.1815 - val_accuracy: 0.5165
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1766 - accuracy: 0.5186 - val_loss: 1.1692 - val_accuracy: 0.5212
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1630 - accuracy: 0.5203 - val_loss: 1.1670 - val_accuracy: 0.5247
Epoch 6/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1538 - accuracy: 0.5250 - val_loss: 1.1682 - val_accuracy: 0.5216
Epoch 7/1000
120000/120000 [====

In [90]:
model5 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model5.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model5.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 13us/sample - loss: 1.2930 - accuracy: 0.4740 - val_loss: 1.1976 - val_accuracy: 0.5134
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2040 - accuracy: 0.5046 - val_loss: 1.1800 - val_accuracy: 0.5182
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1853 - accuracy: 0.5137 - val_loss: 1.1729 - val_accuracy: 0.5201
Epoch 4/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1721 - accuracy: 0.5186 - val_loss: 1.1771 - val_accuracy: 0.5190
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1631 - accuracy: 0.5195 - val_loss: 1.1690 - val_accuracy: 0.5244
Epoch 6/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1563 - accuracy: 0.5236 - val_loss: 1.1703 - val_accuracy: 0.5247
Epoch 7/1000
120000/120000 [====

In [91]:
model6 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model6.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model6.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 14us/sample - loss: 1.2860 - accuracy: 0.4771 - val_loss: 1.1983 - val_accuracy: 0.5124
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2021 - accuracy: 0.5062 - val_loss: 1.1923 - val_accuracy: 0.5175
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1859 - accuracy: 0.5118 - val_loss: 1.1751 - val_accuracy: 0.5204
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1713 - accuracy: 0.5184 - val_loss: 1.1688 - val_accuracy: 0.5214
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1660 - accuracy: 0.5202 - val_loss: 1.1671 - val_accuracy: 0.5232
Epoch 6/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1557 - accuracy: 0.5253 - val_loss: 1.1602 - val_accuracy: 0.5221
Epoch 7/1000
120000/120000 [===

In [92]:
model7 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model7.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model7.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 14us/sample - loss: 1.2899 - accuracy: 0.4717 - val_loss: 1.1929 - val_accuracy: 0.5090
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2027 - accuracy: 0.5056 - val_loss: 1.1800 - val_accuracy: 0.5172
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1840 - accuracy: 0.5141 - val_loss: 1.1750 - val_accuracy: 0.5214
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1723 - accuracy: 0.5174 - val_loss: 1.1762 - val_accuracy: 0.5196
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1613 - accuracy: 0.5229 - val_loss: 1.1698 - val_accuracy: 0.5234
Epoch 6/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1548 - accuracy: 0.5259 - val_loss: 1.1739 - val_accuracy: 0.5193
Epoch 7/1000
120000/120000 [===

In [93]:
model8 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model8.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model8.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 14us/sample - loss: 1.2908 - accuracy: 0.4766 - val_loss: 1.1975 - val_accuracy: 0.5047
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2042 - accuracy: 0.5065 - val_loss: 1.1811 - val_accuracy: 0.5183
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1839 - accuracy: 0.5130 - val_loss: 1.1677 - val_accuracy: 0.5185
Epoch 4/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1734 - accuracy: 0.5169 - val_loss: 1.1724 - val_accuracy: 0.5209
Epoch 5/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1649 - accuracy: 0.5195 - val_loss: 1.1728 - val_accuracy: 0.5232
Epoch 6/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1558 - accuracy: 0.5231 - val_loss: 1.1637 - val_accuracy: 0.5260
Epoch 7/1000
120000/120000 [======

In [94]:
model9 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model9.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model9.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 13us/sample - loss: 1.2875 - accuracy: 0.4755 - val_loss: 1.1945 - val_accuracy: 0.5144
Epoch 2/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.2027 - accuracy: 0.5052 - val_loss: 1.1797 - val_accuracy: 0.5165
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1842 - accuracy: 0.5137 - val_loss: 1.1764 - val_accuracy: 0.5203
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1724 - accuracy: 0.5181 - val_loss: 1.1704 - val_accuracy: 0.5241
Epoch 5/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1661 - accuracy: 0.5199 - val_loss: 1.1699 - val_accuracy: 0.5212
Epoch 6/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1567 - accuracy: 0.5242 - val_loss: 1.1631 - val_accuracy: 0.5213
Epoch 7/1000
120000/120000 [=====

In [95]:
model10 = tf.keras.models.Sequential([
     tf.keras.layers.Dense(512, input_shape=[1175,]),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(512),
     tf.keras.layers.ReLU(),
     tf.keras.layers.Dropout(0.4),
     tf.keras.layers.Dense(6, activation="softmax")
])

model10.compile(loss="sparse_categorical_crossentropy",
               optimizer='adam',
               metrics=["accuracy"])

history = model10.fit(X_train_scaled[:, :-557], y_train, batch_size=512, epochs=1000, callbacks=[early_stop],
                     validation_data=(X_valid_scaled[:, :-557], y_valid))

Train on 120000 samples, validate on 30000 samples
Epoch 1/1000
120000/120000 [==============================] - 2s 13us/sample - loss: 1.2838 - accuracy: 0.4745 - val_loss: 1.1907 - val_accuracy: 0.5142
Epoch 2/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.2023 - accuracy: 0.5058 - val_loss: 1.1769 - val_accuracy: 0.5170
Epoch 3/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1862 - accuracy: 0.5131 - val_loss: 1.1721 - val_accuracy: 0.5176
Epoch 4/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1715 - accuracy: 0.5184 - val_loss: 1.1686 - val_accuracy: 0.5201
Epoch 5/1000
120000/120000 [==============================] - 1s 10us/sample - loss: 1.1621 - accuracy: 0.5195 - val_loss: 1.1705 - val_accuracy: 0.5208
Epoch 6/1000
120000/120000 [==============================] - 1s 9us/sample - loss: 1.1539 - accuracy: 0.5249 - val_loss: 1.1696 - val_accuracy: 0.5246
Epoch 7/1000
120000/120000 [=====

In [96]:
pred1 = model1.predict(X_valid_scaled[:, :-557])
pred2 = model2.predict(X_valid_scaled[:, :-557])
pred3 = model3.predict(X_valid_scaled[:, :-557])
pred4 = model4.predict(X_valid_scaled[:, :-557])
pred5 = model5.predict(X_valid_scaled[:, :-557])
pred6 = model6.predict(X_valid_scaled[:, :-557])
pred7 = model7.predict(X_valid_scaled[:, :-557])
pred8 = model8.predict(X_valid_scaled[:, :-557])
pred9 = model9.predict(X_valid_scaled[:, :-557])
pred10 = model10.predict(X_valid_scaled[:, :-557])

In [97]:
pred_mlp = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10) / 10
log_loss(y_valid, pred_mlp)  # 1.1524238256888075

1.1524238256888075

In [98]:
import lightgbm as lgb

In [99]:
bagging_fraction = 0.7  # 0.7
feature_fraction = 0.9  # 0.9
n_model = 10  # 10

In [100]:
model_dict = {}

for i in range(n_model):
    print('Training %d model'%(i+1))
    d_train = lgb.Dataset(X_train_scaled, label=y_train)
    params = {}
    params['learning_rate'] = 0.1
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'multiclass'
    params['num_classes'] = 6
    params['metric'] = 'multi_logloss'
    params['bagging_fraction'] = bagging_fraction
    params['bagging_freq'] = 1
    params['bagging_seed'] = i+1
    params['feature_fraction'] = feature_fraction
    params['feature_fraction_seed'] = i+1
    params['max_depth'] = 10
    params['num_leaves'] = 32
    params['lambda_l2'] = 24
    #params['max_bin'] = 64

    clf = lgb.train(params, d_train, 100)
    model_dict['model_'+str(i)] = clf  # 1.1645216634935185

Training 1 model
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.835240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 438357
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 1732
[LightGBM] [Info] Start training from score -2.135518
[LightGBM] [Info] Start training from score -0.918084
[LightGBM] [Info] Start training from score -1.060618
[LightGBM] [Info] Start training from score -4.059943
[LightGBM] [Info] Start training from score -2.937149
[LightGBM] [Info] Start training from score -2.717722
Training 2 model
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.719457 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 438357
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 1732
[LightGBM] [Info] Start training from score -2.1

In [101]:
pred_lgb = np.zeros((len(y_valid), 6))
for i in range(n_model):
    pred_lgb += model_dict['model_'+str(i)].predict(X_valid_scaled)
pred_lgb = pred_lgb / n_model   

log_loss(y_valid, pred_lgb)  # 1.1590863715913093

1.1590863715913093

In [102]:
pred = (pred_mlp + pred_lgb) / 2

In [103]:
pred

array([[3.57072546e-03, 5.57289383e-02, 8.97353582e-01, 2.22444551e-04,
        1.85221898e-03, 4.12720756e-02],
       [6.92599978e-02, 2.74608884e-01, 6.21422029e-01, 1.17425804e-03,
        7.18916246e-03, 2.63456698e-02],
       [2.28711285e-01, 2.70365329e-01, 1.99727329e-01, 1.52601798e-01,
        8.85579810e-02, 6.00362813e-02],
       ...,
       [6.39873077e-02, 3.29615641e-01, 4.69737381e-01, 2.69650596e-03,
        3.71115569e-02, 9.68516209e-02],
       [1.86162125e-01, 3.25978530e-01, 3.96424325e-01, 1.27466881e-02,
        3.73194014e-02, 4.13689147e-02],
       [5.89294910e-01, 3.01848296e-01, 8.59463698e-02, 7.33122963e-03,
        7.73586397e-03, 7.84330762e-03]])

In [104]:
log_loss(y_valid, pred)  # 1.147216886499985

1.147216886499985

# test

In [181]:
clnt_id = df_test['CLNT_ID'].unique()

In [182]:
df_test['PD_BUY_AM'] = df_test['PD_BUY_AM'].astype('string')
df_test['PD_BUY_CT'] = df_test['PD_BUY_CT'].astype('string')
df_test['TOT_SESS_HR_V'] = df_test['TOT_SESS_HR_V'].astype('string')

df_test['PD_BUY_AM'] = df_test['PD_BUY_AM'].map(lambda x: x.replace(',', ''))
df_test['PD_BUY_CT'] = df_test['PD_BUY_CT'].map(lambda x: x.replace(',', ''))
df_test['TOT_SESS_HR_V'] = df_test['TOT_SESS_HR_V'].map(lambda x: x.replace(',', ''))

In [183]:
df_test['PD_BUY_AM'] = df_test['PD_BUY_AM'].astype('int')
df_test['PD_BUY_CT'] = df_test['PD_BUY_CT'].astype('int')
df_test['TOT_SESS_HR_V'] = df_test['TOT_SESS_HR_V'].astype('int')

In [184]:
df_test['SESS_DT'] = df_test['SESS_DT'].map(lambda x: date.fromisoformat(str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:]))

In [185]:
num_shoppings = []
avg_prices = []
total_prices = []
avg_cts = []
total_cts = []
avg_sess_views = []
total_sess_views = []
avg_sess_hrs = []
total_sess_hrs = []
avg_shopping_intervals = []
main_devices = []
pd_cs = []
clac1_nms = []
clac2_nms = []
clac3_nms = []

for i in tqdm(range(len(clnt_id))):
    temp_df_test = df_test[df_test['CLNT_ID'] == clnt_id[i]]
    temp_df_test = temp_df_test.sort_values(by=['SESS_DT', 'HITS_SEQ', 'PD_C'])
    temp_df_test = temp_df_test[~temp_df_test.duplicated(subset=['SESS_ID', 'HITS_SEQ', 'PD_C'], keep='last')]
    
    num_shopping = len(temp_df_test)
    avg_price, total_price, avg_ct, total_ct = calc_avg_total_price_ct(temp_df_test)
    avg_sess_view = temp_df_test['TOT_PAG_VIEW_CT'].values.mean()
    total_sess_view = temp_df_test['TOT_PAG_VIEW_CT'].values.sum()
    avg_sess_hr = temp_df_test['TOT_SESS_HR_V'].values.mean()
    total_sess_hr = temp_df_test['TOT_SESS_HR_V'].values.sum()
    avg_shopping_interval = calc_avg_shopping_interval(temp_df_test)
    main_device = scipy.stats.mode(temp_df_test['DVC_CTG_NM'].values).mode[0]
    pd_c = temp_df_test['PD_C'].values
    clac1_nm = temp_df_test['CLAC1_NM'].values
    clac2_nm = temp_df_test['CLAC2_NM'].values
    clac3_nm = temp_df_test['CLAC3_NM'].values
    
    num_shoppings.append(num_shopping)
    avg_prices.append(avg_price)
    total_prices.append(total_price)
    avg_cts.append(avg_ct)
    total_cts.append(total_ct)
    avg_sess_views.append(avg_sess_view)
    total_sess_views.append(total_sess_view)
    avg_sess_hrs.append(avg_sess_hr)
    total_sess_hrs.append(total_sess_hr)
    avg_shopping_intervals.append(avg_shopping_interval)
    main_devices.append(main_device)
    pd_cs.append(pd_c)
    clac1_nms.append(clac1_nm)
    clac2_nms.append(clac2_nm)
    clac3_nms.append(clac3_nm)

100%|█████████████████████████████████████████████████████████████████████████| 113104/113104 [13:29<00:00, 139.73it/s]


In [186]:
data_test = pd.DataFrame([clnt_id, num_shoppings, avg_prices, total_prices, avg_cts, total_cts, avg_sess_views, 
                          total_sess_views, avg_sess_hrs, total_sess_hrs, avg_shopping_intervals, main_devices, 
                          pd_cs, clac1_nms, clac2_nms, clac3_nms]).T

In [187]:
data_test.columns = ['clnt_id', 'num_shopping', 'avg_price', 'total_price', 'avg_ct', 'total_ct', 'avg_sess_view', 
                    'total_sess_view', 'avg_sess_hr', 'total_sess_hr', 'avg_shopping_interval', 'main_device', 
                    'pd_c', 'clac1_nm', 'clac2_nm', 'clac3_nm']

In [188]:
data_test

,clnt_id,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,avg_shopping_interval,main_device,pd_c,clac1_nm,clac2_nm,clac3_nm
0,2,3,32000,128000,1.33333,4,211.333,634,3315,9945,40.5,mobile,"[234664, 325761, 752670]","[스포츠패션, 스포츠패션, 출산/육아용품]","[유아동스포츠화, 유아동스포츠화, 유아스킨/바디케어]","[유아동런닝/트레이닝화, 유아동스포츠샌들/슬리퍼, 유아용화장품]"
1,3,2,106400,212800,1,2,93,186,1051,2102,0,mobile,"[567605, 556431]","[패션잡화, 패션잡화]","[여성지갑, 여성지갑]","[여성일반지갑, 여성일반지갑]"
2,10,3,123000,369000,1,3,134.667,404,1745.67,5237,9.5,mobile,"[269142, 269142, 797432]","[남성의류, 남성의류, 패션잡화]","[남성의류하의, 남성의류하의, 남성가방]","[남성캐주얼바지, 남성캐주얼바지, 남성서류가방]"
3,15,17,17587.9,334170,1.11765,19,105.412,1792,1356.71,23064,7.25,mobile,"[738180, 514099, 566303, 843952, 843603, 76426...","[모바일, 여성의류, 식기/조리기구, 화장품/뷰티케어, 출산/육아용품, 문구/사무용...","[모바일액세서리, 여성의류상의, 밀폐/보관용기, 메이크업, 유아위생용품, 일반문구/...","[기타모바일액세서리, 여성티셔츠/탑, 보온병/텀블러, BB/파운데이션/컴팩트류, 유..."
4,29,2,63000,126000,1,2,69,138,2120,4240,0,mobile,"[674855, 674855]","[화장품/뷰티케어, 화장품/뷰티케어]","[메이크업, 메이크업]","[BB/파운데이션/컴팩트류, BB/파운데이션/컴팩트류]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113099,263089,3,68633.3,205900,1,3,63.3333,190,1700.33,5101,6,mobile,"[50185, 760443, 9617]","[화장품/뷰티케어, 화장품/뷰티케어, 원예/애완]","[스킨케어, 메이크업, 고양이용품]","[스킨케어세트, BB/파운데이션/컴팩트류, 고양이사료]"
113100,263097,4,60610,242440,1,4,469.75,1879,3422,13688,17,mobile,"[737465, 537882, 755981, 301542]","[화장품/뷰티케어, 구기/필드스포츠, 건강식품, 스포츠패션]","[스킨케어, 골프, 홍삼/인삼가공식품, 여성일반스포츠의류]","[스킨케어세트, 골프패션잡화, 홍삼/인삼혼합세트, 여성스포츠티셔츠/탑]"
113101,263098,1,100200,100200,1,1,136,136,1175,1175,183,mobile,[593872],[남성의류],[남성의류상의],[남성남방셔츠]
113102,263099,4,12250,49000,1,4,274,1096,3890,15560,0,mobile,"[193132, 470807, 665269, 692330]","[남성의류, 남성의류, 남성의류, 남성의류]","[남성의류하의, 남성의류상의, 남성의류하의, 남성의류상의]","[남성캐주얼바지, 남성티셔츠, 남성캐주얼바지, 남성티셔츠]"


In [189]:
for nm in df_test['CLAC3_NM'].unique():
    if nm not in clac3_nm_dic.keys():
        clac3_nm_dic[nm] = len(clac3_nm_dic.keys())

In [190]:
clac3_nm_dic

{'블랜더': 0,
 '거실수예소품': 1,
 '남성수영복': 2,
 '페이셜클렌저': 3,
 '선크림류': 4,
 '여성비치웨어': 5,
 '홍삼액': 6,
 'BB/파운데이션/컴팩트류': 7,
 '에센스/세럼': 8,
 '여성속옷세트': 9,
 '브래지어': 10,
 '여성팬티': 11,
 '여성스포츠샌들/슬리퍼': 12,
 '유아동팬티': 13,
 '유아동일반양말': 14,
 '여성샌들': 15,
 '유아동타이즈': 16,
 '주방칼/가위': 17,
 '영유아점프수트/오버롤': 18,
 '아동모': 19,
 '프라이팬': 20,
 '롤티슈': 21,
 '플라스틱서랍장': 22,
 '유아용화장품': 23,
 '일반형냉장고': 24,
 '핸드로션/크림': 25,
 '남성스포츠티셔츠': 26,
 '크림/밤/오일': 27,
 '골프공': 28,
 '골프연습장비': 29,
 '립글로즈/틴트': 30,
 '남성일반지갑': 31,
 '남성런닝/트레이닝화': 32,
 '애견주거/실내용품': 33,
 '생수': 34,
 '여성트레이닝복': 35,
 '여성런닝셔츠/캐미솔': 36,
 '채소즙': 37,
 '남성용스킨케어류': 38,
 '3단우산': 39,
 '스포츠가방': 40,
 '남성팬티': 41,
 '여성남방셔츠': 42,
 '여성원피스': 43,
 '유아동이불/이불커버': 44,
 '남성티셔츠': 45,
 '남성캐주얼바지': 46,
 '여성코트': 47,
 '남성청바지': 48,
 '기타여성속옷': 49,
 '기타냉방가전': 50,
 '영유아티셔츠/탑': 51,
 '여아티셔츠/탑': 52,
 '생리대': 53,
 '유아용기저귀': 54,
 '유산균/프로바이오틱스': 55,
 '김치류': 56,
 '남성등산바지': 57,
 '유아동스포츠샌들/슬리퍼': 58,
 '골프패션잡화': 59,
 '염모제': 60,
 '장우산': 61,
 '골프필드용품': 62,
 '텐트': 63,
 '기타에어컨': 64,
 '여성스니커즈': 65,
 '남성정장셔츠': 66,
 '토스터/제빵기

In [191]:
data_test['clac1_nm'] = data_test['clac1_nm'].map(lambda x: [clac1_nm_dic[nm] for nm in x])
data_test['clac2_nm'] = data_test['clac2_nm'].map(lambda x: [clac2_nm_dic[nm] for nm in x])
data_test['clac3_nm'] = data_test['clac3_nm'].map(lambda x: [clac3_nm_dic[nm] for nm in x])

In [192]:
clac1_matrix = np.zeros((len(data_test), len(clac1_nm_dic)))
for i in range(len(data_test)):
    for j in data_test['clac1_nm'][i]:
        clac1_matrix[i, j] += 1

In [193]:
cols = ['clac1_nm_' + str(i) for i in range(len(clac1_nm_dic))]

In [194]:
clac1_df = pd.DataFrame(clac1_matrix)
clac1_df.columns = cols

In [195]:
clac2_matrix = np.zeros((len(data_test), len(clac2_nm_dic)))
for i in range(len(data_test)):
    for j in data_test['clac2_nm'][i]:
        clac2_matrix[i, j] += 1

In [196]:
cols = ['clac2_nm_' + str(i) for i in range(len(clac2_nm_dic))]

In [197]:
clac2_df = pd.DataFrame(clac2_matrix)
clac2_df.columns = cols

In [198]:
clac3_matrix = np.zeros((len(data_test), len(clac3_nm_dic)))
for i in range(len(data_test)):
    for j in data_test['clac3_nm'][i]:
        clac3_matrix[i, j] += 1

In [199]:
cols = ['clac3_nm_' + str(i) for i in range(len(clac3_nm_dic))]

In [200]:
clac3_df = pd.DataFrame(clac3_matrix)
clac3_df.columns = cols

In [201]:
data_test_concat = pd.concat([data_test.iloc[:, :-5], clac1_df, clac2_df, clac3_df], axis=1)

In [202]:
data_test_concat

,clnt_id,num_shopping,avg_price,total_price,avg_ct,total_ct,avg_sess_view,total_sess_view,avg_sess_hr,total_sess_hr,...,clac3_nm_880,clac3_nm_881,clac3_nm_882,clac3_nm_883,clac3_nm_884,clac3_nm_885,clac3_nm_886,clac3_nm_887,clac3_nm_888,clac3_nm_889
0,2,3,32000,128000,1.33333,4,211.333,634,3315,9945,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,2,106400,212800,1,2,93,186,1051,2102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,3,123000,369000,1,3,134.667,404,1745.67,5237,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15,17,17587.9,334170,1.11765,19,105.412,1792,1356.71,23064,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,29,2,63000,126000,1,2,69,138,2120,4240,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113099,263089,3,68633.3,205900,1,3,63.3333,190,1700.33,5101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113100,263097,4,60610,242440,1,4,469.75,1879,3422,13688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113101,263098,1,100200,100200,1,1,136,136,1175,1175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113102,263099,4,12250,49000,1,4,274,1096,3890,15560,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [203]:
mat_clac1 = np.array(data_test_concat.iloc[:, 11:48]).dot(lookup_table_clac1)
mat_clac1 = mat_clac1 / mat_clac1.sum(1).reshape(-1, 1)

In [204]:
mat_clac1 = pd.DataFrame(mat_clac1)
mat_clac1 = mat_clac1.rename(columns={0: 'given_clac1_F20_prob', 1:'given_clac1_F30_prob', 2:'given_clac1_F40_prob', 3:'given_clac1_M20_prob', 4:'given_clac1_M30_prob', 5:'given_clac1_M40_prob'})

In [205]:
mat_buy_clac1 = np.where(np.array(data_test_concat.iloc[:, 11:48]) >= 1, 1, 0).dot(lookup_table_clac1)
mat_buy_clac1 = mat_buy_clac1 / mat_buy_clac1.sum(1).reshape(-1, 1)

In [206]:
mat_buy_clac1 = pd.DataFrame(mat_buy_clac1)
mat_buy_clac1 = mat_buy_clac1.rename(columns={0: 'buy_clac1_F20_prob', 1:'buy_clac1_F30_prob', 2:'buy_clac1_F40_prob', 3:'buy_clac1_M20_prob', 4:'buy_clac1_M30_prob', 5:'buy_clac1_M40_prob'})

In [207]:
mat_clac2 = np.array(data_test_concat.iloc[:, 48:-890]).dot(lookup_table_clac2)
mat_clac2 = mat_clac2 / mat_clac2.sum(1).reshape(-1, 1)

In [208]:
mat_clac2 = pd.DataFrame(mat_clac2)
mat_clac2 = mat_clac2.rename(columns={0: 'given_clac2_F20_prob', 1:'given_clac2_F30_prob', 2:'given_clac2_F40_prob', 3:'given_clac2_M20_prob', 4:'given_clac2_M30_prob', 5:'given_clac2_M40_prob'})

In [209]:
mat_buy_clac2 = np.where(np.array(data_test_concat.iloc[:, 48:-890]) >= 1, 1, 0).dot(lookup_table_clac2)
mat_buy_clac2 = mat_buy_clac2 / mat_buy_clac2.sum(1).reshape(-1, 1)

In [210]:
mat_buy_clac2 = pd.DataFrame(mat_buy_clac2)
mat_buy_clac2 = mat_buy_clac2.rename(columns={0: 'buy_clac2_F20_prob', 1:'buy_clac2_F30_prob', 2:'buy_clac2_F40_prob', 3:'buy_clac2_M20_prob', 4:'buy_clac2_M30_prob', 5:'buy_clac2_M40_prob'})

In [211]:
lookup_table_clac3 = np.concatenate([lookup_table_clac3, np.ones([7, 6]) * 1/6], axis=0)

In [212]:
mat_clac3 = np.array(data_test_concat.iloc[:, -890:]).dot(lookup_table_clac3)
mat_clac3 = mat_clac3 / mat_clac3.sum(1).reshape(-1, 1)

In [213]:
mat_clac3 = pd.DataFrame(mat_clac3)
mat_clac3 = mat_clac3.rename(columns={0: 'given_clac3_F20_prob', 1:'given_clac3_F30_prob', 2:'given_clac3_F40_prob', 3:'given_clac3_M20_prob', 4:'given_clac3_M30_prob', 5:'given_clac3_M40_prob'})

In [214]:
mat_buy_clac3 = np.where(np.array(data_test_concat.iloc[:, -890:]) >= 1, 1, 0).dot(lookup_table_clac3)
mat_buy_clac3 = mat_buy_clac3 / mat_buy_clac3.sum(1).reshape(-1, 1)

In [215]:
mat_buy_clac3 = pd.DataFrame(mat_buy_clac3)
mat_buy_clac3 = mat_buy_clac3.rename(columns={0: 'buy_clac3_F20_prob', 1:'buy_clac3_F30_prob', 2:'buy_clac3_F40_prob', 3:'buy_clac3_M20_prob', 4:'buy_clac3_M30_prob', 5:'buy_clac3_M40_prob'})

In [216]:
tt = df_test.groupby('CLNT_ID')['PD_C'].value_counts()[df_test['CLNT_ID'].unique()]
kk = df_test['CLNT_ID'].unique()
ww = df_test.groupby(['CLNT_ID', 'PD_C'])['KWD_NM'].count()

In [217]:
pd_mean_vector = []

for words in tqdm(train_pd_w2v[-len(test_pd_w2v):]):
        tmp = np.zeros(256) 
        cnt = 0
        for word in words:
            try:
                tmp += pd_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        pd_mean_vector.append(tmp)
        
pd_mean_vector = np.array(pd_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 113104/113104 [00:04<00:00, 27884.46it/s]


In [218]:
pd_unweight_vector = []

for i, words in tqdm(enumerate(train_pd_w2v[-len(test_pd_w2v):])):
        tmp = np.zeros(256) 
        cnt = 0
        for word in words:
            try:
                tmp += pd_w2v[word] * tt[kk[i]][word]
                cnt += tt[kk[i]][word]
            except:
                pass
        #tmp /= cnt 
        pd_unweight_vector.append(tmp)
        
pd_unweight_vector = np.array(pd_unweight_vector)

113104it [17:48, 105.88it/s]


In [219]:
pd_weight_vector = []

for i, words in tqdm(enumerate(train_pd_w2v[-len(test_pd_w2v):])):
        tmp = np.zeros(256) 
        cnt = 0
        for word in words:
            try:
                tmp += pd_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        pd_weight_vector.append(tmp)
        
pd_weight_vector = np.array(pd_weight_vector)

113104it [09:06, 207.03it/s]


In [220]:
test_pd_mean = pd_mean_vector
test_pd_unweight = pd_unweight_vector
test_pd_weight = pd_weight_vector

test_pd_mean = pd.DataFrame(test_pd_mean)
test_pd_unweight = pd.DataFrame(test_pd_unweight)
test_pd_weight = pd.DataFrame(test_pd_weight)

test_pd_mean.columns = ['pd_mean_'+str(i) for i in range(256)]
test_pd_unweight.columns = ['pd_unweight_'+str(i) for i in range(256)]
test_pd_weight.columns = ['pd_weight_'+str(i) for i in range(256)]

In [221]:
tt = df_test.groupby('CLNT_ID')['PD_BRA_NM'].value_counts()[df_test['CLNT_ID'].unique()]
kk = df_test['CLNT_ID'].unique()
ww = df_test.groupby(['CLNT_ID', 'PD_BRA_NM'])['KWD_NM'].count()

In [222]:
bra_mean_vector = []

for words in tqdm(train_bra_w2v[-len(test_bra_w2v):]):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += bra_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        bra_mean_vector.append(tmp)
        
bra_mean_vector = np.array(bra_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 113104/113104 [00:02<00:00, 39453.33it/s]


In [223]:
bra_unweight_vector = []

for i, words in tqdm(enumerate(train_bra_w2v[-len(test_bra_w2v):])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += bra_w2v[word] * tt[kk[i]][word]
                cnt += tt[kk[i]][word]
            except:
                pass
        #tmp /= cnt 
        bra_unweight_vector.append(tmp)
        
bra_unweight_vector = np.array(bra_unweight_vector)

113104it [10:29, 179.56it/s]


In [224]:
bra_weight_vector = []

for i, words in tqdm(enumerate(train_bra_w2v[-len(test_bra_w2v):])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += bra_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        bra_weight_vector.append(tmp)
        
bra_weight_vector = np.array(bra_weight_vector)

113104it [06:47, 277.46it/s]


In [225]:
test_bra_mean = bra_mean_vector
test_bra_unweight = bra_unweight_vector
test_bra_weight = bra_weight_vector

test_bra_mean = pd.DataFrame(test_bra_mean)
test_bra_unweight = pd.DataFrame(test_bra_unweight)
test_bra_weight = pd.DataFrame(test_bra_weight)

test_bra_mean.columns = ['bra_mean_'+str(i) for i in range(128)]
test_bra_unweight.columns = ['bra_unweight_'+str(i) for i in range(128)]
test_bra_weight.columns = ['bra_weight_'+str(i) for i in range(128)]

In [226]:
tt = df_test.groupby('CLNT_ID')['KWD_NM'].value_counts()[df_test['CLNT_ID'].unique()]
kk = df_test['CLNT_ID'].unique()
ww = df_test.groupby(['CLNT_ID', 'KWD_NM'])['KWD_NM'].count()

In [227]:
kwd_mean_vector = []

for words in tqdm(train_kwd_w2v[-len(test_kwd_w2v):]):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += kwd_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        kwd_mean_vector.append(tmp)
        
kwd_mean_vector = np.array(kwd_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 113104/113104 [00:04<00:00, 23558.67it/s]


In [228]:
kwd_unweight_vector = []

for i, words in tqdm(enumerate(train_kwd_w2v[-len(test_kwd_w2v):])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += kwd_w2v[word] * tt[kk[i]][word]
                cnt += tt[kk[i]][word]
            except:
                pass
        #tmp /= cnt 
        kwd_unweight_vector.append(tmp)
        
kwd_unweight_vector = np.array(kwd_unweight_vector)

113104it [27:14, 69.20it/s] 


In [229]:
kwd_weight_vector = []

for i, words in tqdm(enumerate(train_kwd_w2v[-len(test_kwd_w2v):])):
        tmp = np.zeros(128) 
        cnt = 0
        for word in words:
            try:
                tmp += kwd_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        kwd_weight_vector.append(tmp)
        
kwd_weight_vector = np.array(kwd_weight_vector)

113104it [11:43, 160.87it/s]


In [230]:
test_kwd_mean = kwd_mean_vector
test_kwd_unweight = kwd_unweight_vector
test_kwd_weight = kwd_weight_vector

test_kwd_mean = pd.DataFrame(test_kwd_mean)
test_kwd_unweight = pd.DataFrame(test_kwd_unweight)
test_kwd_weight = pd.DataFrame(test_kwd_weight)

test_kwd_mean.columns = ['kwd_mean_'+str(i) for i in range(128)]
test_kwd_unweight.columns = ['kwd_unweight_'+str(i) for i in range(128)]
test_kwd_weight.columns = ['kwd_weight_'+str(i) for i in range(128)]

In [231]:
tt = df_test.groupby('CLNT_ID')['CLAC3_NM'].value_counts()[df_test['CLNT_ID'].unique()]
kk = df_test['CLNT_ID'].unique()
ww = df_test.groupby(['CLNT_ID', 'CLAC3_NM'])['KWD_NM'].count()

In [232]:
clac3_mean_vector = []

for words in tqdm(train_clac3_w2v[-len(test_clac3_w2v):]):
        tmp = np.zeros(30) 
        cnt = 0
        for word in words:
            try:
                tmp += clac3_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        clac3_mean_vector.append(tmp)
        
clac3_mean_vector = np.array(clac3_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 113104/113104 [00:02<00:00, 41843.84it/s]


In [233]:
clac3_unweight_vector = []

for i, words in tqdm(enumerate(train_clac3_w2v[-len(test_clac3_w2v):])):
        tmp = np.zeros(30) 
        cnt = 0
        for word in words:
            try:
                tmp += clac3_w2v[word] * clac3_matrix[i, clac3_nm_dic[word]]
                cnt += clac3_matrix[i, clac3_nm_dic[word]]
            except:
                pass
        #tmp /= cnt 
        clac3_unweight_vector.append(tmp)
        
clac3_unweight_vector = np.array(clac3_unweight_vector)

113104it [00:04, 26316.69it/s]


In [234]:
clac3_weight_vector = []

for i, words in tqdm(enumerate(train_clac3_w2v[-len(test_clac3_w2v):])):
        tmp = np.zeros(30) 
        cnt = 0
        for word in words:
            try:
                tmp += clac3_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        clac3_weight_vector.append(tmp)
        
clac3_weight_vector = np.array(clac3_weight_vector)

113104it [06:47, 277.63it/s]


In [235]:
test_clac3_mean = clac3_mean_vector
test_clac3_unweight = clac3_unweight_vector
test_clac3_weight = clac3_weight_vector

test_clac3_mean = pd.DataFrame(test_clac3_mean)
test_clac3_unweight = pd.DataFrame(test_clac3_unweight)
test_clac3_weight = pd.DataFrame(test_clac3_weight)

test_clac3_mean.columns = ['clac3_mean_'+str(i) for i in range(30)]
test_clac3_unweight.columns = ['clac3_unweight_'+str(i) for i in range(30)]
test_clac3_weight.columns = ['clac3_weight_'+str(i) for i in range(30)]

In [236]:
tt = df_test.groupby('CLNT_ID')['CLAC2_NM'].value_counts()[df_test['CLNT_ID'].unique()]
kk = df_test['CLNT_ID'].unique()
ww = df_test.groupby(['CLNT_ID', 'CLAC2_NM'])['KWD_NM'].count()

In [237]:
clac2_mean_vector = []

for words in tqdm(train_clac2_w2v[-len(test_clac2_w2v):]):
        tmp = np.zeros(10) 
        cnt = 0
        for word in words:
            try:
                tmp += clac2_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        clac2_mean_vector.append(tmp)
        
clac2_mean_vector = np.array(clac2_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 113104/113104 [00:02<00:00, 46386.39it/s]


In [238]:
clac2_unweight_vector = []

for i, words in tqdm(enumerate(train_clac2_w2v[-len(test_clac2_w2v):])):
        tmp = np.zeros(10) 
        cnt = 0
        for word in words:
            try:
                tmp += clac2_w2v[word] * clac2_matrix[i, clac2_nm_dic[word]]
                cnt += clac2_matrix[i, clac2_nm_dic[word]]
            except:
                pass
        #tmp /= cnt 
        clac2_unweight_vector.append(tmp)
        
clac2_unweight_vector = np.array(clac2_unweight_vector)

113104it [00:03, 31331.93it/s]


In [239]:
clac2_weight_vector = []

for i, words in tqdm(enumerate(train_clac2_w2v[-len(test_clac2_w2v):])):
        tmp = np.zeros(10) 
        cnt = 0
        for word in words:
            try:
                tmp += clac2_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        clac2_weight_vector.append(tmp)
        
clac2_weight_vector = np.array(clac2_weight_vector)

113104it [05:47, 325.31it/s]


In [240]:
test_clac2_mean = clac2_mean_vector
test_clac2_unweight = clac2_unweight_vector
test_clac2_weight = clac2_weight_vector

test_clac2_mean = pd.DataFrame(test_clac2_mean)
test_clac2_unweight = pd.DataFrame(test_clac2_unweight)
test_clac2_weight = pd.DataFrame(test_clac2_weight)

test_clac2_mean.columns = ['clac2_mean_'+str(i) for i in range(10)]
test_clac2_unweight.columns = ['clac2_unweight_'+str(i) for i in range(10)]
test_clac2_weight.columns = ['clac2_weight_'+str(i) for i in range(10)]

In [241]:
tt = df_test.groupby('CLNT_ID')['CLAC1_NM'].value_counts()[df_test['CLNT_ID'].unique()]
kk = df_test['CLNT_ID'].unique()
ww = df_test.groupby(['CLNT_ID', 'CLAC1_NM'])['KWD_NM'].count()

In [242]:
clac1_mean_vector = []

for words in tqdm(train_clac1_w2v[-len(test_clac1_w2v):]):
        tmp = np.zeros(5) 
        cnt = 0
        for word in words:
            try:
                tmp += clac1_w2v[word]
                cnt += 1
            except:
                pass
        #tmp /= cnt  
        clac1_mean_vector.append(tmp)
        
clac1_mean_vector = np.array(clac1_mean_vector)

100%|███████████████████████████████████████████████████████████████████████| 113104/113104 [00:01<00:00, 57800.50it/s]


In [243]:
clac1_unweight_vector = []

for i, words in tqdm(enumerate(train_clac1_w2v[-len(test_clac1_w2v):])):
        tmp = np.zeros(5) 
        cnt = 0
        for word in words:
            try:
                tmp += clac1_w2v[word] * clac1_matrix[i, clac1_nm_dic[word]]
                cnt += clac1_matrix[i, clac1_nm_dic[word]]
            except:
                pass
        #tmp /= cnt 
        clac1_unweight_vector.append(tmp)
        
clac1_unweight_vector = np.array(clac1_unweight_vector)

113104it [00:02, 38104.20it/s]


In [244]:
clac1_weight_vector = []

for i, words in tqdm(enumerate(train_clac1_w2v[-len(test_clac1_w2v):])):
        tmp = np.zeros(5) 
        cnt = 0
        for word in words:
            try:
                tmp += clac1_w2v[word] * (ww[kk[i]] / ww[kk[i]].sum())[word]
                cnt += (ww[kk[i]] / ww[kk[i]].sum())[word]
            except:
                pass
        #tmp /= cnt 
        clac1_weight_vector.append(tmp)
        
clac1_weight_vector = np.array(clac1_weight_vector)

113104it [04:28, 421.84it/s]


In [245]:
test_clac1_mean = clac1_mean_vector
test_clac1_unweight = clac1_unweight_vector
test_clac1_weight = clac1_weight_vector

test_clac1_mean = pd.DataFrame(test_clac1_mean)
test_clac1_unweight = pd.DataFrame(test_clac1_unweight)
test_clac1_weight = pd.DataFrame(test_clac1_weight)

test_clac1_mean.columns = ['clac1_mean_'+str(i) for i in range(5)]
test_clac1_unweight.columns = ['clac1_unweight_'+str(i) for i in range(5)]
test_clac1_weight.columns = ['clac1_weight_'+str(i) for i in range(5)]

In [246]:
data_test_concat = pd.concat([data_test_concat.iloc[:, 1:11], mat_clac1, mat_buy_clac1, mat_clac2, mat_buy_clac2, mat_clac3, mat_buy_clac3, test_pd_mean, test_bra_mean, test_kwd_mean, test_clac1_mean, test_clac2_mean, test_clac3_mean, test_pd_unweight, test_bra_unweight, test_kwd_unweight, test_clac1_unweight, test_clac2_unweight, test_clac3_unweight, test_pd_weight, test_bra_weight, test_kwd_weight, test_clac1_weight, test_clac2_weight, test_clac3_weight], axis=1)

In [247]:
data_test_concat.to_csv('test.csv', index=False)

In [105]:
data_test_concat = pd.read_csv('test.csv')
test_stat = pd.read_csv('stat_test_df.csv')

In [110]:
data_test_concat = pd.concat([test_stat.iloc[:, 1:], data_test_concat.iloc[:, 10:]], axis=1)

In [111]:
X_test_scaled = scaler.transform(data_test_concat)

In [112]:
pred1 = model1.predict(X_test_scaled[:, :-557])
pred2 = model2.predict(X_test_scaled[:, :-557])
pred3 = model3.predict(X_test_scaled[:, :-557])
pred4 = model4.predict(X_test_scaled[:, :-557])
pred5 = model5.predict(X_test_scaled[:, :-557])
pred6 = model6.predict(X_test_scaled[:, :-557])
pred7 = model7.predict(X_test_scaled[:, :-557])
pred8 = model8.predict(X_test_scaled[:, :-557])
pred9 = model9.predict(X_test_scaled[:, :-557])
pred10 = model10.predict(X_test_scaled[:, :-557])

In [113]:
pred_mlp = (pred1 + pred2 + pred3 + pred4 + pred5 + pred6 + pred7 + pred8 + pred9 + pred10) / 10

In [114]:
pred_lgb = np.zeros((len(X_test_scaled), 6))
for i in range(n_model):
    pred_lgb += model_dict['model_'+str(i)].predict(X_test_scaled)
pred_lgb = pred_lgb / n_model   

In [115]:
pred_lgb

array([[0.01483922, 0.75846819, 0.12915512, 0.00084974, 0.06176788,
        0.03491985],
       [0.49494485, 0.2411636 , 0.16966314, 0.05907518, 0.01124558,
        0.02390764],
       [0.04323738, 0.2232262 , 0.49311356, 0.01315682, 0.09265073,
        0.13461531],
       ...,
       [0.05330672, 0.37161363, 0.31219931, 0.02595297, 0.09278512,
        0.14414225],
       [0.17001405, 0.47227626, 0.16736071, 0.05017716, 0.09524455,
        0.04492727],
       [0.0470015 , 0.20116348, 0.6817618 , 0.00733369, 0.01636128,
        0.04637825]])

In [116]:
pred = (pred_mlp + pred_lgb) / 2

In [117]:
pred

array([[0.03059318, 0.72713347, 0.1353143 , 0.00091057, 0.07431102,
        0.03173749],
       [0.57986506, 0.20258786, 0.12928684, 0.06264269, 0.01032955,
        0.01528801],
       [0.04360476, 0.2405963 , 0.4654439 , 0.01737814, 0.09299009,
        0.13998683],
       ...,
       [0.04565929, 0.35647236, 0.34295776, 0.01862472, 0.08863805,
        0.14764782],
       [0.13791403, 0.45997391, 0.23494042, 0.0298647 , 0.08869868,
        0.04860827],
       [0.05066791, 0.18960662, 0.67846334, 0.00921454, 0.01420041,
        0.05784716]])

In [119]:
y_test = pd.read_csv('./sample_submission.csv')

In [120]:
y_test.iloc[:, 1:] = pred

In [121]:
y_test

,CLNT_ID,F20,F30,F40,M20,M30,M40
0,2,0.030593,0.727133,0.135314,0.000911,0.074311,0.031737
1,3,0.579865,0.202588,0.129287,0.062643,0.010330,0.015288
2,10,0.043605,0.240596,0.465444,0.017378,0.092990,0.139987
3,15,0.017765,0.550447,0.280376,0.001969,0.073255,0.076189
4,29,0.552116,0.329472,0.091156,0.007910,0.011378,0.007968
...,...,...,...,...,...,...,...
113099,263089,0.173268,0.391064,0.402903,0.002203,0.011206,0.019356
113100,263097,0.143175,0.268077,0.522866,0.004085,0.011733,0.050065
113101,263098,0.045659,0.356472,0.342958,0.018625,0.088638,0.147648
113102,263099,0.137914,0.459974,0.234940,0.029865,0.088699,0.048608


In [122]:
y_test.to_csv('./sample_submission.csv', index=False)